# Preparación de Datos
**Proyecto Machine Learning:** Limpieza y Preparación de Datos de Nacimientos y Defunciones

Este notebook implementa la limpieza y preparación de los datasets identificados en el análisis exploratorio, enfocándose en:
1. Limpieza crítica del dataset `defunciones_filtradas`
2. Estandarización de nombres de columnas
3. Validación de rangos de edad
4. Integración de datasets
5. Validación de consistencia
6. Preparación para modelado


In [1]:
# 1. Importación de librerías y configuración

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Configuración general
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Opciones de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Librerías importadas correctamente")


✅ Librerías importadas correctamente


In [2]:
# 2. Carga de datos

# Definir rutas de los archivos
ruta_por_sexo = r"C:\ProyectoML2\proyecto-ml\data\01_raw\dataset_nacimiento-defuncion_por_sexo.csv"
ruta_defunciones_filtradas = r"C:\ProyectoML2\proyecto-ml\data\01_raw\datos_filtrados_2014_2023.csv"
ruta_por_edad_madre = r"C:\ProyectoML2\proyecto-ml\data\01_raw\nacimiento_rango_edad_madre.csv"
ruta_por_edad_fallecido = r"C:\ProyectoML2\proyecto-ml\data\01_raw\rango_edad_fallecido.csv"
ruta_setdedatos = r"C:\ProyectoML2\proyecto-ml\data\01_raw\setdedatos.csv"

# Función para cargar CSV con diferentes separadores
def cargar_csv(ruta):
    """
    Carga un archivo CSV probando primero con ',' y luego con ';' como separador
    """
    try:
        return pd.read_csv(ruta)
    except:
        return pd.read_csv(ruta, sep=";")

# Cargar todos los datasets
por_sexo = cargar_csv(ruta_por_sexo)
defunciones_filtradas = cargar_csv(ruta_defunciones_filtradas)
por_edad_madre = cargar_csv(ruta_por_edad_madre)
por_edad_fallecido = cargar_csv(ruta_por_edad_fallecido)
setdedatos = cargar_csv(ruta_setdedatos)

print("✅ Archivos cargados correctamente")
print(f"📊 Dataset defunciones_filtradas: {defunciones_filtradas.shape[0]:,} registros")


✅ Archivos cargados correctamente
📊 Dataset defunciones_filtradas: 1,250,062 registros


## 3. Limpieza Crítica del Dataset `defunciones_filtradas`

Este es el dataset más problemático identificado en el EDA. Requiere limpieza antes de cualquier análisis posterior.


In [3]:
# 3.1 Análisis inicial del dataset defunciones_filtradas

print("=== ANÁLISIS INICIAL DE DEFUNCIONES_FILTRADAS ===")
print(f"📊 Dimensiones: {defunciones_filtradas.shape}")
print(f"📋 Columnas: {list(defunciones_filtradas.columns)}")
print("\n=== VALORES NULOS POR COLUMNA ===")
nulos_por_columna = defunciones_filtradas.isnull().sum()
print(nulos_por_columna[nulos_por_columna > 0])

print("\n=== DUPLICADOS ===")
duplicados = defunciones_filtradas.duplicated().sum()
print(f"Registros duplicados: {duplicados:,}")

print("\n=== PRIMERAS FILAS ===")
display(defunciones_filtradas.head())


=== ANÁLISIS INICIAL DE DEFUNCIONES_FILTRADAS ===
📊 Dimensiones: (1250062, 10)
📋 Columnas: ['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_TIPO', 'EDAD_CANT', 'COD_COMUNA', 'COMUNA', 'NOMBRE_REGION', 'CAPITULO_DIAG1', 'GLOSA_CAPITULO_DIAG1']

=== VALORES NULOS POR COLUMNA ===
FECHA_DEF        19
COD_COMUNA        4
COMUNA            4
NOMBRE_REGION     4
dtype: int64

=== DUPLICADOS ===
Registros duplicados: 3,844

=== PRIMERAS FILAS ===


,AÑO,FECHA_DEF,SEXO_NOMBRE,EDAD_TIPO,EDAD_CANT,COD_COMUNA,COMUNA,NOMBRE_REGION,CAPITULO_DIAG1,GLOSA_CAPITULO_DIAG1
0,2015,2015-01-11,Mujer,2.0,4,13127.0,Recoleta,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ..."
1,2016,2016-01-31,Hombre,1.0,20,8203.0,Cañete,Del Bíobío,S00-T98,"Traumatismos, envenenamientos y algunas otras ..."
2,2019,2019-08-08,Hombre,1.0,18,13303.0,Tiltil,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ..."
3,2015,2015-02-17,Hombre,1.0,19,13119.0,Maipú,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ..."
4,2015,2015-01-03,Hombre,1.0,18,13115.0,Lo Barnechea,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ..."


In [4]:
# 3.2 Eliminación de registros duplicados

print("=== ELIMINACIÓN DE DUPLICADOS ===")
print(f"Registros antes de eliminar duplicados: {defunciones_filtradas.shape[0]:,}")

# Crear una copia del dataset para trabajar
defunciones_limpio = defunciones_filtradas.copy()

# Eliminar duplicados manteniendo la primera ocurrencia
defunciones_limpio = defunciones_limpio.drop_duplicates(keep='first')

print(f"Registros después de eliminar duplicados: {defunciones_limpio.shape[0]:,}")
print(f"Registros eliminados: {defunciones_filtradas.shape[0] - defunciones_limpio.shape[0]:,}")

# Verificar que no quedan duplicados
duplicados_restantes = defunciones_limpio.duplicated().sum()
print(f"Duplicados restantes: {duplicados_restantes}")


=== ELIMINACIÓN DE DUPLICADOS ===
Registros antes de eliminar duplicados: 1,250,062
Registros después de eliminar duplicados: 1,246,218
Registros eliminados: 3,844
Duplicados restantes: 0


In [5]:
# 3.3 Manejo de valores nulos en información geográfica

print("=== MANEJO DE VALORES NULOS EN INFORMACIÓN GEOGRÁFICA ===")

# Verificar valores nulos en columnas geográficas críticas
columnas_geograficas = ['COD_COMUNA', 'COMUNA', 'NOMBRE_REGION']
nulos_geograficos = defunciones_limpio[columnas_geograficas].isnull().sum()
print("Valores nulos en columnas geográficas:")
print(nulos_geograficos)

# Mostrar registros con valores nulos en información geográfica
registros_nulos_geo = defunciones_limpio[defunciones_limpio[columnas_geograficas].isnull().any(axis=1)]
print(f"\nRegistros con valores nulos en información geográfica: {len(registros_nulos_geo)}")

if len(registros_nulos_geo) > 0:
    print("\nEjemplos de registros con valores nulos:")
    display(registros_nulos_geo[['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_CANT'] + columnas_geograficas].head())

# Eliminar registros con valores nulos en información geográfica crítica
# La información geográfica es esencial para análisis regionales
defunciones_limpio = defunciones_limpio.dropna(subset=columnas_geograficas)

print(f"\nRegistros después de eliminar nulos geográficos: {defunciones_limpio.shape[0]:,}")
print(f"Registros eliminados por nulos geográficos: {len(registros_nulos_geo)}")


=== MANEJO DE VALORES NULOS EN INFORMACIÓN GEOGRÁFICA ===
Valores nulos en columnas geográficas:
COD_COMUNA       4
COMUNA           4
NOMBRE_REGION    4
dtype: int64

Registros con valores nulos en información geográfica: 4

Ejemplos de registros con valores nulos:


,AÑO,FECHA_DEF,SEXO_NOMBRE,EDAD_CANT,COD_COMUNA,COMUNA,NOMBRE_REGION
1159454,2024,2024-02-28,Hombre,83,NaN,NaN,NaN
1166415,2024,2024-04-26,Mujer,52,NaN,NaN,NaN
1193568,2024,2024-04-11,Hombre,1,NaN,NaN,NaN
1248759,2024,2024-06-26,Mujer,2,NaN,NaN,NaN



Registros después de eliminar nulos geográficos: 1,246,214
Registros eliminados por nulos geográficos: 4


In [6]:
# 3.4 Manejo de valores nulos en FECHA_DEF

print("=== MANEJO DE VALORES NULOS EN FECHA_DEF ===")

# Verificar valores nulos en FECHA_DEF
nulos_fecha = defunciones_limpio['FECHA_DEF'].isnull().sum()
print(f"Valores nulos en FECHA_DEF: {nulos_fecha}")

if nulos_fecha > 0:
    # Mostrar registros con valores nulos en FECHA_DEF
    registros_nulos_fecha = defunciones_limpio[defunciones_limpio['FECHA_DEF'].isnull()]
    print(f"\nRegistros con valores nulos en FECHA_DEF: {len(registros_nulos_fecha)}")
    print("\nEjemplos de registros con fecha nula:")
    display(registros_nulos_fecha[['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_CANT', 'COMUNA']].head())
    
    # Estrategia: Imputar con fecha media del año correspondiente
    # Esto mantiene la información temporal aproximada
    print("\n=== IMPUTACIÓN DE FECHAS NULAS ===")
    
    for año in registros_nulos_fecha['AÑO'].unique():
        # Calcular fecha media del año para registros con fecha válida
        fechas_validas_año = defunciones_limpio[
            (defunciones_limpio['AÑO'] == año) & 
            (defunciones_limpio['FECHA_DEF'].notnull())
        ]['FECHA_DEF']
        
        if len(fechas_validas_año) > 0:
            # Convertir a datetime para calcular media
            fechas_datetime = pd.to_datetime(fechas_validas_año, errors='coerce')
            fecha_media = fechas_datetime.mean()
            
            # Imputar fecha media en registros nulos del año
            mask_nulos_año = (defunciones_limpio['AÑO'] == año) & (defunciones_limpio['FECHA_DEF'].isnull())
            defunciones_limpio.loc[mask_nulos_año, 'FECHA_DEF'] = fecha_media.strftime('%Y-%m-%d')
            
            print(f"Año {año}: {mask_nulos_año.sum()} registros imputados con fecha {fecha_media.strftime('%Y-%m-%d')}")
    
    # Verificar que no quedan valores nulos
    nulos_restantes = defunciones_limpio['FECHA_DEF'].isnull().sum()
    print(f"\nValores nulos restantes en FECHA_DEF: {nulos_restantes}")

else:
    print("No hay valores nulos en FECHA_DEF")


=== MANEJO DE VALORES NULOS EN FECHA_DEF ===
Valores nulos en FECHA_DEF: 19

Registros con valores nulos en FECHA_DEF: 19

Ejemplos de registros con fecha nula:


,AÑO,FECHA_DEF,SEXO_NOMBRE,EDAD_CANT,COMUNA
5893,2016,NaN,Hombre,33,Temuco
7953,2015,NaN,Hombre,44,Temuco
8144,2014,NaN,Hombre,35,Ercilla
39420,2016,NaN,Hombre,62,Gorbea
55107,2014,NaN,Hombre,81,Temuco



=== IMPUTACIÓN DE FECHAS NULAS ===
Año 2016: 3 registros imputados con fecha 2016-07-03
Año 2015: 6 registros imputados con fecha 2015-07-05
Año 2014: 3 registros imputados con fecha 2014-07-04
Año 2018: 1 registros imputados con fecha 2018-07-05
Año 2019: 1 registros imputados con fecha 2019-07-04
Año 2020: 3 registros imputados con fecha 2020-07-05
Año 2021: 2 registros imputados con fecha 2021-06-27

Valores nulos restantes en FECHA_DEF: 0


In [7]:
# 3.5 Estandarización del formato de fechas

print("=== ESTANDARIZACIÓN DEL FORMATO DE FECHAS ===")

# Convertir FECHA_DEF a datetime para análisis temporal
print("Convirtiendo FECHA_DEF a formato datetime...")

# Convertir a datetime, manejando posibles errores de formato
defunciones_limpio['FECHA_DEF'] = pd.to_datetime(defunciones_limpio['FECHA_DEF'], errors='coerce')

# Verificar conversión
fechas_invalidas = defunciones_limpio['FECHA_DEF'].isnull().sum()
print(f"Fechas que no se pudieron convertir: {fechas_invalidas}")

if fechas_invalidas > 0:
    print("Registros con fechas inválidas:")
    registros_fecha_invalida = defunciones_limpio[defunciones_limpio['FECHA_DEF'].isnull()]
    display(registros_fecha_invalida[['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_CANT']].head())
    
    # Eliminar registros con fechas inválidas (son muy pocos)
    defunciones_limpio = defunciones_limpio.dropna(subset=['FECHA_DEF'])
    print(f"Registros eliminados por fechas inválidas: {fechas_invalidas}")

# Crear variables temporales derivadas para análisis
print("\n=== CREACIÓN DE VARIABLES TEMPORALES ===")

# Extraer año, mes, día de la semana, trimestre
defunciones_limpio['AÑO_FECHA'] = defunciones_limpio['FECHA_DEF'].dt.year
defunciones_limpio['MES'] = defunciones_limpio['FECHA_DEF'].dt.month
defunciones_limpio['DIA_SEMANA'] = defunciones_limpio['FECHA_DEF'].dt.day_name()
defunciones_limpio['TRIMESTRE'] = defunciones_limpio['FECHA_DEF'].dt.quarter
defunciones_limpio['DIA_AÑO'] = defunciones_limpio['FECHA_DEF'].dt.dayofyear

print("Variables temporales creadas:")
print("- AÑO_FECHA: Año de la fecha de defunción")
print("- MES: Mes (1-12)")
print("- DIA_SEMANA: Día de la semana")
print("- TRIMESTRE: Trimestre (1-4)")
print("- DIA_AÑO: Día del año (1-365/366)")

# Verificar consistencia entre AÑO y AÑO_FECHA
inconsistencias_año = (defunciones_limpio['AÑO'] != defunciones_limpio['AÑO_FECHA']).sum()
print(f"\nInconsistencias entre AÑO y AÑO_FECHA: {inconsistencias_año}")

if inconsistencias_año > 0:
    print("Ejemplos de inconsistencias:")
    inconsistencias = defunciones_limpio[defunciones_limpio['AÑO'] != defunciones_limpio['AÑO_FECHA']]
    display(inconsistencias[['AÑO', 'FECHA_DEF', 'AÑO_FECHA']].head())


=== ESTANDARIZACIÓN DEL FORMATO DE FECHAS ===
Convirtiendo FECHA_DEF a formato datetime...
Fechas que no se pudieron convertir: 0

=== CREACIÓN DE VARIABLES TEMPORALES ===
Variables temporales creadas:
- AÑO_FECHA: Año de la fecha de defunción
- MES: Mes (1-12)
- DIA_SEMANA: Día de la semana
- TRIMESTRE: Trimestre (1-4)
- DIA_AÑO: Día del año (1-365/366)

Inconsistencias entre AÑO y AÑO_FECHA: 0


In [8]:
# 3.6 Resumen de la limpieza del dataset defunciones_filtradas

print("=== RESUMEN DE LA LIMPIEZA DE DEFUNCIONES_FILTRADAS ===")
print(f"📊 Registros originales: {defunciones_filtradas.shape[0]:,}")
print(f"📊 Registros después de limpieza: {defunciones_limpio.shape[0]:,}")
print(f"📊 Registros eliminados: {defunciones_filtradas.shape[0] - defunciones_limpio.shape[0]:,}")
print(f"📊 Porcentaje de datos conservados: {(defunciones_limpio.shape[0] / defunciones_filtradas.shape[0]) * 100:.2f}%")

print("\n=== VERIFICACIÓN FINAL ===")
print("Valores nulos por columna:")
nulos_finales = defunciones_limpio.isnull().sum()
print(nulos_finales[nulos_finales > 0])

print(f"\nDuplicados restantes: {defunciones_limpio.duplicated().sum()}")

print("\n=== INFORMACIÓN DEL DATASET LIMPIO ===")
print(f"Columnas: {defunciones_limpio.shape[1]}")
print(f"Rango de fechas: {defunciones_limpio['FECHA_DEF'].min()} a {defunciones_limpio['FECHA_DEF'].max()}")
print(f"Años cubiertos: {sorted(defunciones_limpio['AÑO'].unique())}")

print("\n✅ Dataset defunciones_filtradas limpiado exitosamente")


=== RESUMEN DE LA LIMPIEZA DE DEFUNCIONES_FILTRADAS ===
📊 Registros originales: 1,250,062
📊 Registros después de limpieza: 1,246,214
📊 Registros eliminados: 3,848
📊 Porcentaje de datos conservados: 99.69%

=== VERIFICACIÓN FINAL ===
Valores nulos por columna:
Series([], dtype: int64)

Duplicados restantes: 0

=== INFORMACIÓN DEL DATASET LIMPIO ===
Columnas: 15
Rango de fechas: 2014-01-01 00:00:00 a 2024-09-28 00:00:00
Años cubiertos: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

✅ Dataset defunciones_filtradas limpiado exitosamente


In [9]:
# 3.7 Guardar dataset limpio para uso posterior

print("=== GUARDANDO DATASET LIMPIO ===")

# Crear la carpeta 02_intermediate si no existe
import os
carpeta_intermediate = r"C:\ProyectoML2\proyecto-ml\data\02_intermediate"
if not os.path.exists(carpeta_intermediate):
    os.makedirs(carpeta_intermediate)
    print(f"📁 Carpeta creada: {carpeta_intermediate}")

# Guardar el dataset limpio en formato CSV
ruta_guardado = r"C:\ProyectoML2\proyecto-ml\data\02_intermediate\defunciones_limpias.csv"
defunciones_limpio.to_csv(ruta_guardado, index=False)

print(f"✅ Dataset guardado exitosamente en: {ruta_guardado}")
print(f"📊 Tamaño del archivo: {os.path.getsize(ruta_guardado) / (1024*1024):.2f} MB")

# Verificar que el archivo se guardó correctamente
if os.path.exists(ruta_guardado):
    print("✅ Verificación: Archivo guardado correctamente")
    
    # Cargar una muestra para verificar
    muestra_verificacion = pd.read_csv(ruta_guardado, nrows=5)
    print("\n📋 Muestra del archivo guardado:")
    display(muestra_verificacion)
else:
    print("❌ Error: No se pudo guardar el archivo")


=== GUARDANDO DATASET LIMPIO ===
✅ Dataset guardado exitosamente en: C:\ProyectoML2\proyecto-ml\data\02_intermediate\defunciones_limpias.csv
📊 Tamaño del archivo: 163.54 MB
✅ Verificación: Archivo guardado correctamente

📋 Muestra del archivo guardado:


,AÑO,FECHA_DEF,SEXO_NOMBRE,EDAD_TIPO,EDAD_CANT,COD_COMUNA,COMUNA,NOMBRE_REGION,CAPITULO_DIAG1,GLOSA_CAPITULO_DIAG1,AÑO_FECHA,MES,DIA_SEMANA,TRIMESTRE,DIA_AÑO
0,2015,2015-01-11,Mujer,2.0,4,13127.0,Recoleta,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ...",2015,1,Sunday,1,11
1,2016,2016-01-31,Hombre,1.0,20,8203.0,Cañete,Del Bíobío,S00-T98,"Traumatismos, envenenamientos y algunas otras ...",2016,1,Sunday,1,31
2,2019,2019-08-08,Hombre,1.0,18,13303.0,Tiltil,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ...",2019,8,Thursday,3,220
3,2015,2015-02-17,Hombre,1.0,19,13119.0,Maipú,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ...",2015,2,Tuesday,1,48
4,2015,2015-01-03,Hombre,1.0,18,13115.0,Lo Barnechea,Metropolitana de Santiago,S00-T98,"Traumatismos, envenenamientos y algunas otras ...",2015,1,Saturday,1,3


## 4. Estandarización de Nombres de Columnas

Esta sección se enfoca en unificar y estandarizar los nombres de columnas entre todos los datasets para mantener consistencia en el proyecto.


In [10]:
# 4.1 Análisis de nombres de columnas en todos los datasets

print("=== ANÁLISIS DE NOMBRES DE COLUMNAS ===")

# Crear diccionario con todos los datasets para análisis
datasets_para_estandarizar = {
    "por_sexo": por_sexo,
    "defunciones_limpio": defunciones_limpio,
    "por_edad_madre": por_edad_madre,
    "por_edad_fallecido": por_edad_fallecido,
    "setdedatos": setdedatos
}

# Mostrar nombres de columnas de cada dataset
for nombre_dataset, df in datasets_para_estandarizar.items():
    print(f"\n📋 {nombre_dataset.upper()}:")
    print(f"   Columnas: {list(df.columns)}")
    print(f"   Dimensiones: {df.shape}")

print("\n=== PROBLEMAS IDENTIFICADOS ===")
print("1. Inconsistencia en 'Año' vs 'año'")
print("2. Espacios inconsistentes en nombres (ej: 'Defuncion(Hombre)' vs 'Defuncion (Mujer)')")
print("3. Nombres de regiones con/sin tildes")
print("4. Nombres de columnas muy largos o poco descriptivos")


=== ANÁLISIS DE NOMBRES DE COLUMNAS ===

📋 POR_SEXO:
   Columnas: ['Año', 'Nacimiento (Hombre)', 'Nacimiento (Mujer)', 'Defuncion(Hombre)', 'Defuncion (Mujer)']
   Dimensiones: (9, 5)

📋 DEFUNCIONES_LIMPIO:
   Columnas: ['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_TIPO', 'EDAD_CANT', 'COD_COMUNA', 'COMUNA', 'NOMBRE_REGION', 'CAPITULO_DIAG1', 'GLOSA_CAPITULO_DIAG1', 'AÑO_FECHA', 'MES', 'DIA_SEMANA', 'TRIMESTRE', 'DIA_AÑO']
   Dimensiones: (1246214, 15)

📋 POR_EDAD_MADRE:
   Columnas: ['Año', 'Menores de 15 años', '15 a 19 años', '20 a 24 años', '25 a 29 años', '30 a 34 años', '35 a 39 años', '40 a 44 años', '45 a 49 años', '50 y más años']
   Dimensiones: (14, 10)

📋 POR_EDAD_FALLECIDO:
   Columnas: ['Año', 'Menores de 1 año', '1 a 4', '5 a 9', '10 a 14', '15 a 19', '20 a 24', '25 a 29', '30 a 34', '35 a 39', '40 a 44', '45 a 49', '50 o mas']
   Dimensiones: (14, 13)

📋 SETDEDATOS:
   Columnas: ['año', 'Nacimientos', 'Defunciones']
   Dimensiones: (50, 3)

=== PROBLEMAS IDENTIFICADOS ===
1

In [11]:
# 4.2 Estandarización de nombres de columnas - Dataset por_sexo

print("=== ESTANDARIZACIÓN: POR_SEXO ===")

# Crear copia para trabajar
por_sexo_estandarizado = por_sexo.copy()

print("Columnas originales:")
print(list(por_sexo_estandarizado.columns))

# Definir mapeo de nombres de columnas
mapeo_por_sexo = {
    'Año': 'año',
    'Nacimiento (Hombre)': 'nacimientos_hombres',
    'Nacimiento (Mujer)': 'nacimientos_mujeres', 
    'Defuncion(Hombre)': 'defunciones_hombres',
    'Defuncion (Mujer)': 'defunciones_mujeres'
}

# Renombrar columnas
por_sexo_estandarizado = por_sexo_estandarizado.rename(columns=mapeo_por_sexo)

print("\nColumnas estandarizadas:")
print(list(por_sexo_estandarizado.columns))

print("\n✅ Dataset por_sexo estandarizado")


=== ESTANDARIZACIÓN: POR_SEXO ===
Columnas originales:
['Año', 'Nacimiento (Hombre)', 'Nacimiento (Mujer)', 'Defuncion(Hombre)', 'Defuncion (Mujer)']

Columnas estandarizadas:
['año', 'nacimientos_hombres', 'nacimientos_mujeres', 'defunciones_hombres', 'defunciones_mujeres']

✅ Dataset por_sexo estandarizado


In [12]:
# 4.3 Estandarización de nombres de columnas - Dataset defunciones_limpio

print("=== ESTANDARIZACIÓN: DEFUNCIONES_LIMPIO ===")

# Crear copia para trabajar
defunciones_estandarizado = defunciones_limpio.copy()

print("Columnas originales:")
print(list(defunciones_estandarizado.columns))

# Definir mapeo de nombres de columnas
mapeo_defunciones = {
    'AÑO': 'año',
    'FECHA_DEF': 'fecha_defuncion',
    'SEXO_NOMBRE': 'sexo',
    'EDAD_TIPO': 'tipo_edad',
    'EDAD_CANT': 'edad_cantidad',
    'COD_COMUNA': 'codigo_comuna',
    'COMUNA': 'comuna',
    'NOMBRE_REGION': 'region',
    'CAPITULO_DIAG1': 'codigo_diagnostico',
    'GLOSA_CAPITULO_DIAG1': 'descripcion_diagnostico',
    'AÑO_FECHA': 'año_fecha',
    'MES': 'mes',
    'DIA_SEMANA': 'dia_semana',
    'TRIMESTRE': 'trimestre',
    'DIA_AÑO': 'dia_año'
}

# Renombrar columnas
defunciones_estandarizado = defunciones_estandarizado.rename(columns=mapeo_defunciones)

print("\nColumnas estandarizadas:")
print(list(defunciones_estandarizado.columns))

print("\n✅ Dataset defunciones_limpio estandarizado")


=== ESTANDARIZACIÓN: DEFUNCIONES_LIMPIO ===
Columnas originales:
['AÑO', 'FECHA_DEF', 'SEXO_NOMBRE', 'EDAD_TIPO', 'EDAD_CANT', 'COD_COMUNA', 'COMUNA', 'NOMBRE_REGION', 'CAPITULO_DIAG1', 'GLOSA_CAPITULO_DIAG1', 'AÑO_FECHA', 'MES', 'DIA_SEMANA', 'TRIMESTRE', 'DIA_AÑO']

Columnas estandarizadas:
['año', 'fecha_defuncion', 'sexo', 'tipo_edad', 'edad_cantidad', 'codigo_comuna', 'comuna', 'region', 'codigo_diagnostico', 'descripcion_diagnostico', 'año_fecha', 'mes', 'dia_semana', 'trimestre', 'dia_año']

✅ Dataset defunciones_limpio estandarizado


In [13]:
# 4.4 Estandarización de nombres de columnas - Dataset por_edad_madre

print("=== ESTANDARIZACIÓN: POR_EDAD_MADRE ===")

# Crear copia para trabajar
por_edad_madre_estandarizado = por_edad_madre.copy()

print("Columnas originales:")
print(list(por_edad_madre_estandarizado.columns))

# Definir mapeo de nombres de columnas
mapeo_edad_madre = {
    'Año': 'año',
    'Menores de 15 años': 'nacimientos_menores_15',
    '15 a 19 años': 'nacimientos_15_19',
    '20 a 24 años': 'nacimientos_20_24',
    '25 a 29 años': 'nacimientos_25_29',
    '30 a 34 años': 'nacimientos_30_34',
    '35 a 39 años': 'nacimientos_35_39',
    '40 a 44 años': 'nacimientos_40_44',
    '45 a 49 años': 'nacimientos_45_49',
    '50 y más años': 'nacimientos_50_mas'
}

# Renombrar columnas
por_edad_madre_estandarizado = por_edad_madre_estandarizado.rename(columns=mapeo_edad_madre)

print("\nColumnas estandarizadas:")
print(list(por_edad_madre_estandarizado.columns))

print("\n✅ Dataset por_edad_madre estandarizado")


=== ESTANDARIZACIÓN: POR_EDAD_MADRE ===
Columnas originales:
['Año', 'Menores de 15 años', '15 a 19 años', '20 a 24 años', '25 a 29 años', '30 a 34 años', '35 a 39 años', '40 a 44 años', '45 a 49 años', '50 y más años']

Columnas estandarizadas:
['año', 'nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34', 'nacimientos_35_39', 'nacimientos_40_44', 'nacimientos_45_49', 'nacimientos_50_mas']

✅ Dataset por_edad_madre estandarizado


In [14]:
# 4.5 Estandarización de nombres de columnas - Dataset por_edad_fallecido

print("=== ESTANDARIZACIÓN: POR_EDAD_FALLECIDO ===")

# Crear copia para trabajar
por_edad_fallecido_estandarizado = por_edad_fallecido.copy()

print("Columnas originales:")
print(list(por_edad_fallecido_estandarizado.columns))

# Definir mapeo de nombres de columnas
mapeo_edad_fallecido = {
    'Año': 'año',
    'Menores de 1 año': 'defunciones_menores_1',
    '1 a 4': 'defunciones_1_4',
    '5 a 9': 'defunciones_5_9',
    '10 a 14': 'defunciones_10_14',
    '15 a 19': 'defunciones_15_19',
    '20 a 24': 'defunciones_20_24',
    '25 a 29': 'defunciones_25_29',
    '30 a 34': 'defunciones_30_34',
    '35 a 39': 'defunciones_35_39',
    '40 a 44': 'defunciones_40_44',
    '45 a 49': 'defunciones_45_49',
    '50 o mas': 'defunciones_50_mas'
}

# Renombrar columnas
por_edad_fallecido_estandarizado = por_edad_fallecido_estandarizado.rename(columns=mapeo_edad_fallecido)

print("\nColumnas estandarizadas:")
print(list(por_edad_fallecido_estandarizado.columns))

print("\n✅ Dataset por_edad_fallecido estandarizado")


=== ESTANDARIZACIÓN: POR_EDAD_FALLECIDO ===
Columnas originales:
['Año', 'Menores de 1 año', '1 a 4', '5 a 9', '10 a 14', '15 a 19', '20 a 24', '25 a 29', '30 a 34', '35 a 39', '40 a 44', '45 a 49', '50 o mas']

Columnas estandarizadas:
['año', 'defunciones_menores_1', 'defunciones_1_4', 'defunciones_5_9', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 'defunciones_45_49', 'defunciones_50_mas']

✅ Dataset por_edad_fallecido estandarizado


In [15]:
# 4.6 Estandarización de nombres de columnas - Dataset setdedatos

print("=== ESTANDARIZACIÓN: SETDEDATOS ===")

# Crear copia para trabajar
setdedatos_estandarizado = setdedatos.copy()

print("Columnas originales:")
print(list(setdedatos_estandarizado.columns))

# Definir mapeo de nombres de columnas
mapeo_setdedatos = {
    'año': 'año',  # Ya está en minúsculas
    'Nacimientos': 'nacimientos_totales',
    'Defunciones': 'defunciones_totales'
}

# Renombrar columnas
setdedatos_estandarizado = setdedatos_estandarizado.rename(columns=mapeo_setdedatos)

print("\nColumnas estandarizadas:")
print(list(setdedatos_estandarizado.columns))

print("\n✅ Dataset setdedatos estandarizado")


=== ESTANDARIZACIÓN: SETDEDATOS ===
Columnas originales:
['año', 'Nacimientos', 'Defunciones']

Columnas estandarizadas:
['año', 'nacimientos_totales', 'defunciones_totales']

✅ Dataset setdedatos estandarizado


In [16]:
# 4.7 Estandarización de nombres de regiones

print("=== ESTANDARIZACIÓN DE NOMBRES DE REGIONES ===")

# Analizar nombres únicos de regiones en el dataset de defunciones
regiones_unicas = defunciones_estandarizado['region'].unique()
print("Regiones únicas encontradas:")
for i, region in enumerate(sorted(regiones_unicas), 1):
    print(f"{i:2d}. {region}")

# Definir mapeo para estandarizar nombres de regiones
mapeo_regiones = {
    'Del Bíobío': 'Del Biobío',  # Quitar tilde
    'Metropolitana de Santiago': 'Región Metropolitana',
    'De Tarapacá': 'De Tarapacá',  # Ya está correcto
    'De Antofagasta': 'De Antofagasta',  # Ya está correcto
    'De Atacama': 'De Atacama',  # Ya está correcto
    'De Coquimbo': 'De Coquimbo',  # Ya está correcto
    'De Valparaíso': 'De Valparaíso',  # Ya está correcto
    'Del Libertador B. O\'Higgins': 'Del Libertador General Bernardo O\'Higgins',
    'Del Maule': 'Del Maule',  # Ya está correcto
    'De Ñuble': 'De Ñuble',  # Ya está correcto
    'De La Araucanía': 'De La Araucanía',  # Ya está correcto
    'De Los Ríos': 'De Los Ríos',  # Ya está correcto
    'De Los Lagos': 'De Los Lagos',  # Ya está correcto
    'De Aysén del General Carlos Ibáñez del Campo': 'De Aysén del General Carlos Ibáñez del Campo',  # Ya está correcto
    'De Magallanes y de la Antártica Chilena': 'De Magallanes y de la Antártica Chilena'  # Ya está correcto
}

# Aplicar mapeo de regiones
defunciones_estandarizado['region'] = defunciones_estandarizado['region'].replace(mapeo_regiones)

print("\nRegiones después de estandarización:")
regiones_estandarizadas = defunciones_estandarizado['region'].unique()
for i, region in enumerate(sorted(regiones_estandarizadas), 1):
    print(f"{i:2d}. {region}")

print("\n✅ Nombres de regiones estandarizados")


=== ESTANDARIZACIÓN DE NOMBRES DE REGIONES ===
Regiones únicas encontradas:
 1. De Aisén del Gral. C. Ibáñez del Campo
 2. De Antofagasta
 3. De Arica y Parinacota
 4. De Atacama
 5. De Coquimbo
 6. De La Araucanía
 7. De Los Lagos
 8. De Los Ríos
 9. De Magallanes y de La Antártica Chilena
10. De Tarapacá
11. De Valparaíso
12. De Ñuble
13. Del Bíobío
14. Del Libertador B. O'Higgins
15. Del Maule
16. Ignorada
17. Metropolitana de Santiago

Regiones después de estandarización:
 1. De Aisén del Gral. C. Ibáñez del Campo
 2. De Antofagasta
 3. De Arica y Parinacota
 4. De Atacama
 5. De Coquimbo
 6. De La Araucanía
 7. De Los Lagos
 8. De Los Ríos
 9. De Magallanes y de La Antártica Chilena
10. De Tarapacá
11. De Valparaíso
12. De Ñuble
13. Del Biobío
14. Del Libertador General Bernardo O'Higgins
15. Del Maule
16. Ignorada
17. Región Metropolitana

✅ Nombres de regiones estandarizados


In [17]:
# 4.8 Resumen de la estandarización de nombres de columnas

print("=== RESUMEN DE LA ESTANDARIZACIÓN DE NOMBRES DE COLUMNAS ===")

# Crear diccionario con todos los datasets estandarizados
datasets_estandarizados = {
    "por_sexo_estandarizado": por_sexo_estandarizado,
    "defunciones_estandarizado": defunciones_estandarizado,
    "por_edad_madre_estandarizado": por_edad_madre_estandarizado,
    "por_edad_fallecido_estandarizado": por_edad_fallecido_estandarizado,
    "setdedatos_estandarizado": setdedatos_estandarizado
}

print("📋 Nombres de columnas estandarizados:")
for nombre_dataset, df in datasets_estandarizados.items():
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Columnas: {list(df.columns)}")

print("\n=== BENEFICIOS DE LA ESTANDARIZACIÓN ===")
print("✅ Todos los datasets usan 'año' en minúsculas")
print("✅ Nombres de columnas más descriptivos y consistentes")
print("✅ Espacios y caracteres especiales estandarizados")
print("✅ Nombres de regiones unificados")
print("✅ Facilita la integración y análisis posterior")

print("\n=== DATASETS ESTANDARIZADOS DISPONIBLES ===")
print("Los siguientes datasets están listos para análisis:")
for nombre in datasets_estandarizados.keys():
    print(f"  - {nombre}")

print("\n✅ Estandarización de nombres de columnas completada")


=== RESUMEN DE LA ESTANDARIZACIÓN DE NOMBRES DE COLUMNAS ===
📋 Nombres de columnas estandarizados:

POR_SEXO_ESTANDARIZADO:
   Columnas: ['año', 'nacimientos_hombres', 'nacimientos_mujeres', 'defunciones_hombres', 'defunciones_mujeres']

DEFUNCIONES_ESTANDARIZADO:
   Columnas: ['año', 'fecha_defuncion', 'sexo', 'tipo_edad', 'edad_cantidad', 'codigo_comuna', 'comuna', 'region', 'codigo_diagnostico', 'descripcion_diagnostico', 'año_fecha', 'mes', 'dia_semana', 'trimestre', 'dia_año']

POR_EDAD_MADRE_ESTANDARIZADO:
   Columnas: ['año', 'nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34', 'nacimientos_35_39', 'nacimientos_40_44', 'nacimientos_45_49', 'nacimientos_50_mas']

POR_EDAD_FALLECIDO_ESTANDARIZADO:
   Columnas: ['año', 'defunciones_menores_1', 'defunciones_1_4', 'defunciones_5_9', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 

## 5. Validación de Rangos de Edad

Esta sección se enfoca en verificar la consistencia de los rangos de edad entre datasets y validar que los valores de edad sean lógicos.


In [18]:
# 5.1 Análisis de rangos de edad en datasets de nacimientos

print("=== ANÁLISIS DE RANGOS DE EDAD - NACIMIENTOS ===")

# Analizar rangos de edad en dataset por_edad_madre
print("📋 Rangos de edad en nacimientos (por_edad_madre_estandarizado):")
columnas_edad_madre = [col for col in por_edad_madre_estandarizado.columns if col != 'año']
print("Columnas de edad:", columnas_edad_madre)

# Mostrar estadísticas básicas de cada rango
print("\n📊 Estadísticas por rango de edad de madre:")
for col in columnas_edad_madre:
    min_val = por_edad_madre_estandarizado[col].min()
    max_val = por_edad_madre_estandarizado[col].max()
    mean_val = por_edad_madre_estandarizado[col].mean()
    print(f"  {col:25s}: Min={min_val:6.0f}, Max={max_val:6.0f}, Promedio={mean_val:6.0f}")

# Verificar consistencia en el formato de nombres
print("\n🔍 Verificación de formato de nombres:")
for col in columnas_edad_madre:
    if 'mas' in col or 'más' in col:
        print(f"  ⚠️  Posible inconsistencia en: {col}")
    else:
        print(f"  ✅ Formato consistente: {col}")


=== ANÁLISIS DE RANGOS DE EDAD - NACIMIENTOS ===
📋 Rangos de edad en nacimientos (por_edad_madre_estandarizado):
Columnas de edad: ['nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34', 'nacimientos_35_39', 'nacimientos_40_44', 'nacimientos_45_49', 'nacimientos_50_mas']

📊 Estadísticas por rango de edad de madre:
  nacimientos_menores_15   : Min=   158, Max=   963, Promedio=   558
  nacimientos_15_19        : Min=  6428, Max= 38047, Promedio= 20607
  nacimientos_20_24        : Min= 28334, Max= 59884, Promedio= 46498
  nacimientos_25_29        : Min= 45178, Max= 63210, Promedio= 57008
  nacimientos_30_34        : Min= 50523, Max= 57520, Promedio= 54143
  nacimientos_35_39        : Min= 30989, Max= 34942, Promedio= 32793
  nacimientos_40_44        : Min=  8257, Max=  9545, Promedio=  8869
  nacimientos_45_49        : Min=   441, Max=   602, Promedio=   502
  nacimientos_50_mas       : Min=     5, Max=    32, Promedio=    17

🔍 Verif

In [19]:
# 5.2 Análisis de rangos de edad en datasets de defunciones

print("=== ANÁLISIS DE RANGOS DE EDAD - DEFUNCIONES ===")

# Analizar rangos de edad en dataset por_edad_fallecido
print("📋 Rangos de edad en defunciones (por_edad_fallecido_estandarizado):")
columnas_edad_fallecido = [col for col in por_edad_fallecido_estandarizado.columns if col != 'año']
print("Columnas de edad:", columnas_edad_fallecido)

# Mostrar estadísticas básicas de cada rango
print("\n📊 Estadísticas por rango de edad de fallecido:")
for col in columnas_edad_fallecido:
    min_val = por_edad_fallecido_estandarizado[col].min()
    max_val = por_edad_fallecido_estandarizado[col].max()
    mean_val = por_edad_fallecido_estandarizado[col].mean()
    print(f"  {col:25s}: Min={min_val:6.0f}, Max={max_val:6.0f}, Promedio={mean_val:6.0f}")

# Verificar consistencia en el formato de nombres
print("\n🔍 Verificación de formato de nombres:")
for col in columnas_edad_fallecido:
    if 'mas' in col or 'más' in col:
        print(f"  ⚠️  Posible inconsistencia en: {col}")
    else:
        print(f"  ✅ Formato consistente: {col}")

# Analizar valores de EDAD_CANT en dataset de defunciones detalladas
print("\n📊 Análisis de EDAD_CANT en defunciones detalladas:")
print(f"Valores únicos de edad_cantidad: {sorted(defunciones_estandarizado['edad_cantidad'].unique())}")
print(f"Rango de edad_cantidad: {defunciones_estandarizado['edad_cantidad'].min()} - {defunciones_estandarizado['edad_cantidad'].max()}")
print(f"Valores nulos en edad_cantidad: {defunciones_estandarizado['edad_cantidad'].isnull().sum()}")


=== ANÁLISIS DE RANGOS DE EDAD - DEFUNCIONES ===
📋 Rangos de edad en defunciones (por_edad_fallecido_estandarizado):
Columnas de edad: ['defunciones_menores_1', 'defunciones_1_4', 'defunciones_5_9', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 'defunciones_45_49', 'defunciones_50_mas']

📊 Estadísticas por rango de edad de fallecido:
  defunciones_menores_1    : Min=  1022, Max=  1908, Promedio=  1512
  defunciones_1_4          : Min=   204, Max=   312, Promedio=   253
  defunciones_5_9          : Min=   129, Max=   188, Promedio=   161
  defunciones_10_14        : Min=   162, Max=   238, Promedio=   200
  defunciones_15_19        : Min=   465, Max=   733, Promedio=   574
  defunciones_20_24        : Min=   857, Max=  1065, Promedio=   932
  defunciones_25_29        : Min=   955, Max=  1276, Promedio=  1101
  defunciones_30_34        : Min=  1088, Max=  1618, Promedio=  1255
  defuncion

In [20]:
# 5.3 Comparación de rangos de edad entre datasets

print("=== COMPARACIÓN DE RANGOS DE EDAD ENTRE DATASETS ===")

# Crear mapeo de rangos de edad para comparación
rangos_nacimientos = {
    'nacimientos_menores_15': '0-14',
    'nacimientos_15_19': '15-19', 
    'nacimientos_20_24': '20-24',
    'nacimientos_25_29': '25-29',
    'nacimientos_30_34': '30-34',
    'nacimientos_35_39': '35-39',
    'nacimientos_40_44': '40-44',
    'nacimientos_45_49': '45-49',
    'nacimientos_50_mas': '50+'
}

rangos_defunciones = {
    'defunciones_menores_1': '0-1',
    'defunciones_1_4': '1-4',
    'defunciones_5_9': '5-9',
    'defunciones_10_14': '10-14',
    'defunciones_15_19': '15-19',
    'defunciones_20_24': '20-24',
    'defunciones_25_29': '25-29',
    'defunciones_30_34': '30-34',
    'defunciones_35_39': '35-39',
    'defunciones_40_44': '40-44',
    'defunciones_45_49': '45-49',
    'defunciones_50_mas': '50+'
}

print("📋 Rangos de edad en nacimientos:")
for col, rango in rangos_nacimientos.items():
    print(f"  {col:25s} → {rango}")

print("\n📋 Rangos de edad en defunciones:")
for col, rango in rangos_defunciones.items():
    print(f"  {col:25s} → {rango}")

# Identificar rangos superpuestos
rangos_comunes = set(rangos_nacimientos.values()) & set(rangos_defunciones.values())
print(f"\n🔍 Rangos de edad comunes: {sorted(rangos_comunes)}")

# Identificar rangos únicos
rangos_unicos_nacimientos = set(rangos_nacimientos.values()) - set(rangos_defunciones.values())
rangos_unicos_defunciones = set(rangos_defunciones.values()) - set(rangos_nacimientos.values())

print(f"📊 Rangos únicos en nacimientos: {sorted(rangos_unicos_nacimientos)}")
print(f"📊 Rangos únicos en defunciones: {sorted(rangos_unicos_defunciones)}")


=== COMPARACIÓN DE RANGOS DE EDAD ENTRE DATASETS ===
📋 Rangos de edad en nacimientos:
  nacimientos_menores_15    → 0-14
  nacimientos_15_19         → 15-19
  nacimientos_20_24         → 20-24
  nacimientos_25_29         → 25-29
  nacimientos_30_34         → 30-34
  nacimientos_35_39         → 35-39
  nacimientos_40_44         → 40-44
  nacimientos_45_49         → 45-49
  nacimientos_50_mas        → 50+

📋 Rangos de edad en defunciones:
  defunciones_menores_1     → 0-1
  defunciones_1_4           → 1-4
  defunciones_5_9           → 5-9
  defunciones_10_14         → 10-14
  defunciones_15_19         → 15-19
  defunciones_20_24         → 20-24
  defunciones_25_29         → 25-29
  defunciones_30_34         → 30-34
  defunciones_35_39         → 35-39
  defunciones_40_44         → 40-44
  defunciones_45_49         → 45-49
  defunciones_50_mas        → 50+

🔍 Rangos de edad comunes: ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50+']
📊 Rangos únicos en nacimientos: ['0-1

In [23]:
# 5.4 Validación de valores de edad en dataset detallado (MODIFICADO)

print("=== VALIDACIÓN DE VALORES DE EDAD EN DATASET DETALLADO ===")

# Analizar distribución de valores de edad_cantidad
print("📊 Distribución de edad_cantidad:")
print(defunciones_estandarizado['edad_cantidad'].describe())

# 1. ELIMINAR edades imposibles (>120 años)
print("\n🔍 Eliminando edades imposibles (>120 años):")
edades_imposibles = defunciones_estandarizado[defunciones_estandarizado['edad_cantidad'] > 120]
print(f"Registros con edad > 120 años: {len(edades_imposibles)}")

if len(edades_imposibles) > 0:
    print("Ejemplos de edades imposibles:")
    display(edades_imposibles[['año', 'fecha_defuncion', 'sexo', 'edad_cantidad', 'region']].head())
    
    # Eliminar estos registros
    defunciones_estandarizado = defunciones_estandarizado[defunciones_estandarizado['edad_cantidad'] <= 120]
    print(f"\n✅ Eliminados {len(edades_imposibles)} registros con edad > 120 años")
    print(f"📊 Dataset actualizado: {len(defunciones_estandarizado):,} registros")

# 2. NO detectar outliers por edad (todas las edades son válidas)
print(f"\n✅ TODAS LAS EDADES SON VÁLIDAS")
print(f"📊 Rango de edades: {defunciones_estandarizado['edad_cantidad'].min()} - {defunciones_estandarizado['edad_cantidad'].max()} años")

# 3. Mostrar distribución por grupos de edad
print(f"\n�� Distribución por grupos de edad:")
defunciones_estandarizado['grupo_edad'] = pd.cut(defunciones_estandarizado['edad_cantidad'], 
                                                bins=[0, 1, 5, 18, 65, 120], 
                                                labels=['0-1 años', '2-4 años', '5-17 años', '18-64 años', '65+ años'])
distribucion_grupos = defunciones_estandarizado['grupo_edad'].value_counts().sort_index()
for grupo, cantidad in distribucion_grupos.items():
    porcentaje = (cantidad / len(defunciones_estandarizado)) * 100
    print(f"  {grupo:12s}: {cantidad:6,} ({porcentaje:5.1f}%)")

# 4. Solo validar rangos lógicos básicos
print(f"\n🔍 Validación de rangos lógicos:")
edades_negativas = defunciones_estandarizado[defunciones_estandarizado['edad_cantidad'] < 0]
print(f"Registros con edad negativa: {len(edades_negativas)}")

if len(edades_negativas) > 0:
    print("Ejemplos de edades negativas:")
    display(edades_negativas[['año', 'fecha_defuncion', 'sexo', 'edad_cantidad']].head())

print(f"\n✅ Validación completada: Dataset limpio con edades válidas")


=== VALIDACIÓN DE VALORES DE EDAD EN DATASET DETALLADO ===
📊 Distribución de edad_cantidad:
count    1.246214e+06
mean     7.221377e+01
std      1.880039e+01
min      0.000000e+00
25%      6.300000e+01
50%      7.600000e+01
75%      8.600000e+01
max      9.990000e+02
Name: edad_cantidad, dtype: float64

🔍 Eliminando edades imposibles (>120 años):
Registros con edad > 120 años: 14
Ejemplos de edades imposibles:


,año,fecha_defuncion,sexo,edad_cantidad,region
4545,2014,2014-06-30,Hombre,999,De La Araucanía
24752,2014,2014-04-19,Hombre,999,De Tarapacá
32330,2015,2015-10-30,Hombre,999,De La Araucanía
41877,2018,2018-04-18,Hombre,121,De Los Ríos
43947,2014,2014-03-21,Hombre,999,De Ñuble



✅ Eliminados 14 registros con edad > 120 años
📊 Dataset actualizado: 1,246,200 registros

✅ TODAS LAS EDADES SON VÁLIDAS
📊 Rango de edades: 0 - 118 años

�� Distribución por grupos de edad:
  0-1 años    :  6,358 (  0.5%)
  2-4 años    :  6,135 (  0.5%)
  5-17 años   : 11,417 (  0.9%)
  18-64 años  : 331,208 ( 26.6%)
  65+ años    : 891,071 ( 71.5%)

🔍 Validación de rangos lógicos:
Registros con edad negativa: 0

✅ Validación completada: Dataset limpio con edades válidas


In [24]:
# 5.5 Estandarización final de rangos de edad

print("=== ESTANDARIZACIÓN FINAL DE RANGOS DE EDAD ===")

# Crear datasets finales con rangos de edad estandarizados
print("📋 Creando datasets con rangos de edad estandarizados...")

# Dataset de nacimientos por edad (ya está estandarizado)
nacimientos_edad_final = por_edad_madre_estandarizado.copy()
print("✅ Dataset nacimientos por edad: ya estandarizado")

# Dataset de defunciones por edad (ya está estandarizado)
defunciones_edad_final = por_edad_fallecido_estandarizado.copy()
print("✅ Dataset defunciones por edad: ya estandarizado")

# Crear función para categorizar edad en rangos estándar
def categorizar_edad(edad):
    """
    Categoriza la edad en rangos estándar para análisis
    """
    if pd.isna(edad):
        return 'desconocida'
    elif edad < 0:
        return 'edad_invalida'
    elif edad < 1:
        return 'menores_1'
    elif edad < 5:
        return '1_4'
    elif edad < 10:
        return '5_9'
    elif edad < 15:
        return '10_14'
    elif edad < 20:
        return '15_19'
    elif edad < 25:
        return '20_24'
    elif edad < 30:
        return '25_29'
    elif edad < 35:
        return '30_34'
    elif edad < 40:
        return '35_39'
    elif edad < 45:
        return '40_44'
    elif edad < 50:
        return '45_49'
    elif edad < 55:
        return '50_54'
    elif edad < 60:
        return '55_59'
    elif edad < 65:
        return '60_64'
    elif edad < 70:
        return '65_69'
    elif edad < 75:
        return '70_74'
    elif edad < 80:
        return '75_79'
    elif edad < 85:
        return '80_84'
    elif edad < 90:
        return '85_89'
    elif edad < 95:
        return '90_94'
    elif edad < 100:
        return '95_99'
    else:
        return '100_mas'

# Aplicar categorización al dataset de defunciones detalladas
defunciones_estandarizado['rango_edad'] = defunciones_estandarizado['edad_cantidad'].apply(categorizar_edad)

print("\n�� Distribución de rangos de edad en defunciones detalladas:")
distribucion_rangos = defunciones_estandarizado['rango_edad'].value_counts().sort_index()
print(distribucion_rangos)

# Verificar que no hay valores problemáticos
valores_problematicos = defunciones_estandarizado[defunciones_estandarizado['rango_edad'].isin(['desconocida', 'edad_invalida'])]
print(f"\nRegistros con edad problemática: {len(valores_problematicos)}")

if len(valores_problematicos) > 0:
    print("Ejemplos de registros problemáticos:")
    display(valores_problematicos[['año', 'fecha_defuncion', 'sexo', 'edad_cantidad', 'rango_edad']].head())

=== ESTANDARIZACIÓN FINAL DE RANGOS DE EDAD ===
📋 Creando datasets con rangos de edad estandarizados...
✅ Dataset nacimientos por edad: ya estandarizado
✅ Dataset defunciones por edad: ya estandarizado

�� Distribución de rangos de edad en defunciones detalladas:
rango_edad
100_mas       11429
10_14          3344
15_19          6463
1_4           11459
20_24         10257
25_29         12511
30_34         14136
35_39         16647
40_44         22356
45_49         31723
50_54         47168
55_59         67120
5_9            4362
60_64         87673
65_69        108708
70_74        131625
75_79        152925
80_84        168669
85_89        167254
90_94        121052
95_99         49308
menores_1        11
Name: count, dtype: int64

Registros con edad problemática: 0


In [25]:
# 5.6 Resumen de la validación de rangos de edad

print("=== RESUMEN DE LA VALIDACIÓN DE RANGOS DE EDAD ===")

# Crear diccionario con datasets finales de edad
datasets_edad_finales = {
    "nacimientos_edad_final": nacimientos_edad_final,
    "defunciones_edad_final": defunciones_edad_final,
    "defunciones_estandarizado": defunciones_estandarizado  # Incluye nueva columna rango_edad
}

print("�� Datasets con rangos de edad validados:")
for nombre_dataset, df in datasets_edad_finales.items():
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Dimensiones: {df.shape}")
    columnas_edad = [col for col in df.columns if 'edad' in col.lower() or 'nacimientos_' in col or 'defunciones_' in col]
    print(f"   Columnas relacionadas con edad: {columnas_edad}")

print("\n=== BENEFICIOS DE LA VALIDACIÓN ===")
print("✅ Rangos de edad consistentes entre datasets")
print("✅ Formato estandarizado ('50_mas' en lugar de '50 y más' o '50 o mas')")
print("✅ Validación de valores lógicos de edad")
print("✅ Eliminación de edades imposibles (>120 años)")
print("✅ Preservación de todos los casos de menores de 18 años")
print("✅ Nueva columna 'rango_edad' para análisis categórico")
print("✅ Función de categorización reutilizable con rangos detallados")

print("\n=== RANGOS DE EDAD ESTÁNDAR (ACTUALIZADOS) ===")
rangos_estandar = [
    'menores_1', '1_4', '5_9', '10_14', '15_19', '20_24', 
    '25_29', '30_34', '35_39', '40_44', '45_49', '50_54',
    '55_59', '60_64', '65_69', '70_74', '75_79', '80_84',
    '85_89', '90_94', '95_99', '100_mas'
]
print("Rangos estándar implementados:")
for i, rango in enumerate(rangos_estandar, 1):
    print(f"  {i:2d}. {rango}")

print("\n=== CORRECCIONES APLICADAS ===")
print("�� Eliminados registros con edad > 120 años (errores de captura)")
print("🔧 Preservados todos los casos de menores de 18 años")
print("🔧 Agregados rangos detallados para adultos mayores (50-100+ años)")
print("🔧 Eliminada detección de outliers por edad (todas las edades son válidas)")

print("\n✅ Validación de rangos de edad completada")


=== RESUMEN DE LA VALIDACIÓN DE RANGOS DE EDAD ===
�� Datasets con rangos de edad validados:

NACIMIENTOS_EDAD_FINAL:
   Dimensiones: (14, 10)
   Columnas relacionadas con edad: ['nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34', 'nacimientos_35_39', 'nacimientos_40_44', 'nacimientos_45_49', 'nacimientos_50_mas']

DEFUNCIONES_EDAD_FINAL:
   Dimensiones: (14, 13)
   Columnas relacionadas con edad: ['defunciones_menores_1', 'defunciones_1_4', 'defunciones_5_9', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 'defunciones_45_49', 'defunciones_50_mas']

DEFUNCIONES_ESTANDARIZADO:
   Dimensiones: (1246200, 17)
   Columnas relacionadas con edad: ['tipo_edad', 'edad_cantidad', 'grupo_edad', 'rango_edad']

=== BENEFICIOS DE LA VALIDACIÓN ===
✅ Rangos de edad consistentes entre datasets
✅ Formato estandarizado ('50_mas' en lugar de '50 y m

## 6. Integración de Datasets

Esta sección se enfoca en crear un dataset unificado temporal combinando información de diferentes fuentes y generando variables derivadas útiles para análisis.


In [26]:
# 6.1 Análisis de períodos temporales en todos los datasets

print("=== ANÁLISIS DE PERÍODOS TEMPORALES ===")

# Analizar rangos de años en cada dataset
datasets_temporales = {
    "setdedatos_estandarizado": setdedatos_estandarizado,
    "por_sexo_estandarizado": por_sexo_estandarizado,
    "nacimientos_edad_final": nacimientos_edad_final,
    "defunciones_edad_final": defunciones_edad_final,
    "defunciones_estandarizado": defunciones_estandarizado
}

print("📅 Períodos temporales por dataset:")
for nombre_dataset, df in datasets_temporales.items():
    años = sorted(df['año'].unique())
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Años: {años[0]} - {años[-1]} ({len(años)} años)")
    print(f"   Rango completo: {años}")

# Identificar años comunes y únicos
años_setdedatos = set(setdedatos_estandarizado['año'].unique())
años_por_sexo = set(por_sexo_estandarizado['año'].unique())
años_nacimientos_edad = set(nacimientos_edad_final['año'].unique())
años_defunciones_edad = set(defunciones_edad_final['año'].unique())

print(f"\n🔍 Análisis de cobertura temporal:")
print(f"Años en setdedatos (1974-2023): {len(años_setdedatos)} años")
print(f"Años en por_sexo (2015-2023): {len(años_por_sexo)} años")
print(f"Años en nacimientos_edad (2010-2023): {len(años_nacimientos_edad)} años")
print(f"Años en defunciones_edad (2010-2023): {len(años_defunciones_edad)} años")

# Años comunes para integración
años_comunes = años_setdedatos & años_por_sexo & años_nacimientos_edad & años_defunciones_edad
print(f"\nAños comunes a todos los datasets: {sorted(años_comunes)} ({len(años_comunes)} años)")

# Años únicos en cada dataset
años_unicos_setdedatos = años_setdedatos - años_por_sexo
años_unicos_por_sexo = años_por_sexo - años_setdedatos
print(f"Años únicos en setdedatos: {sorted(años_unicos_setdedatos)}")
print(f"Años únicos en por_sexo: {sorted(años_unicos_por_sexo)}")


=== ANÁLISIS DE PERÍODOS TEMPORALES ===
📅 Períodos temporales por dataset:

SETDEDATOS_ESTANDARIZADO:
   Años: 1974 - 2023 (50 años)
   Rango completo: [np.int64(1974), np.int64(1975), np.int64(1976), np.int64(1977), np.int64(1978), np.int64(1979), np.int64(1980), np.int64(1981), np.int64(1982), np.int64(1983), np.int64(1984), np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]

POR_SEXO_ESTANDARIZADO:
   Años: 2015 - 2023 (

In [27]:
# 6.2 Crear dataset unificado temporal básico

print("=== CREACIÓN DE DATASET UNIFICADO TEMPORAL ===")

# Crear dataset base con información de setdedatos (serie más larga: 1974-2023)
dataset_unificado = setdedatos_estandarizado.copy()
print(f"📊 Dataset base (setdedatos): {dataset_unificado.shape[0]} registros, {dataset_unificado.shape[1]} columnas")

# Agregar información de por_sexo para años 2015-2023
print("\n🔗 Integrando información por sexo...")
dataset_unificado = dataset_unificado.merge(
    por_sexo_estandarizado, 
    on='año', 
    how='left'
)

print(f"📊 Después de integrar por_sexo: {dataset_unificado.shape[0]} registros, {dataset_unificado.shape[1]} columnas")

# Verificar integración
print("\n📋 Columnas del dataset unificado:")
print(list(dataset_unificado.columns))

# Mostrar años con información completa vs parcial
años_completa = dataset_unificado[dataset_unificado['nacimientos_hombres'].notna()]['año'].tolist()
años_parcial = dataset_unificado[dataset_unificado['nacimientos_hombres'].isna()]['año'].tolist()

print(f"\n📅 Años con información completa (2015-2023): {len(años_completa)} años")
print(f"📅 Años con información parcial (1974-2014): {len(años_parcial)} años")

print("\nPrimeras filas del dataset unificado:")
display(dataset_unificado.head(10))


=== CREACIÓN DE DATASET UNIFICADO TEMPORAL ===
📊 Dataset base (setdedatos): 50 registros, 3 columnas

🔗 Integrando información por sexo...
📊 Después de integrar por_sexo: 50 registros, 7 columnas

📋 Columnas del dataset unificado:
['año', 'nacimientos_totales', 'defunciones_totales', 'nacimientos_hombres', 'nacimientos_mujeres', 'defunciones_hombres', 'defunciones_mujeres']

📅 Años con información completa (2015-2023): 9 años
📅 Años con información parcial (1974-2014): 41 años

Primeras filas del dataset unificado:


,año,nacimientos_totales,defunciones_totales,nacimientos_hombres,nacimientos_mujeres,defunciones_hombres,defunciones_mujeres
0,2023,171992,121270,87713.0,84262.0,63174.0,58085.0
1,2022,189310,136958,96011.0,93284.0,71676.0,65275.0
2,2021,177255,137439,90355.0,86883.0,73308.0,64119.0
3,2020,194952,125833,99908.0,95025.0,67453.0,58367.0
4,2019,210188,109658,107353.0,102812.0,57632.0,52010.0
5,2018,221731,106796,113039.0,108668.0,56093.0,50684.0
6,2017,219186,106388,111660.0,107501.0,55773.0,50593.0
7,2016,231749,104026,117801.0,113920.0,54761.0,49239.0
8,2015,244670,103327,124713.0,119936.0,54693.0,48615.0
9,2014,250997,101960,NaN,NaN,NaN,NaN


In [28]:
# 6.3 Generar variables derivadas - Tasas y ratios

print("=== GENERACIÓN DE VARIABLES DERIVADAS ===")

# Crear variables derivadas útiles para análisis
print("📊 Calculando variables derivadas...")

# 1. Tasas de natalidad y mortalidad (por cada 1000 habitantes)
# Nota: Para cálculos precisos necesitaríamos población, pero podemos usar aproximaciones
dataset_unificado['tasa_natalidad'] = (dataset_unificado['nacimientos_totales'] / 1000).round(2)
dataset_unificado['tasa_mortalidad'] = (dataset_unificado['defunciones_totales'] / 1000).round(2)

# 2. Ratio de nacimientos por sexo (hombres/mujeres)
dataset_unificado['ratio_nacimientos_sexo'] = (
    dataset_unificado['nacimientos_hombres'] / dataset_unificado['nacimientos_mujeres']
).round(3)

# 3. Ratio de defunciones por sexo (hombres/mujeres)
dataset_unificado['ratio_defunciones_sexo'] = (
    dataset_unificado['defunciones_hombres'] / dataset_unificado['defunciones_mujeres']
).round(3)

# 4. Crecimiento natural (nacimientos - defunciones)
dataset_unificado['crecimiento_natural'] = (
    dataset_unificado['nacimientos_totales'] - dataset_unificado['defunciones_totales']
)

# 5. Porcentaje de crecimiento natural
dataset_unificado['porcentaje_crecimiento_natural'] = (
    (dataset_unificado['crecimiento_natural'] / dataset_unificado['nacimientos_totales']) * 100
).round(2)

# 6. Diferencia año a año en nacimientos y defunciones
dataset_unificado['dif_nacimientos_año_anterior'] = dataset_unificado['nacimientos_totales'].diff()
dataset_unificado['dif_defunciones_año_anterior'] = dataset_unificado['defunciones_totales'].diff()

# 7. Porcentaje de cambio año a año
dataset_unificado['pct_cambio_nacimientos'] = (
    (dataset_unificado['dif_nacimientos_año_anterior'] / dataset_unificado['nacimientos_totales'].shift(1)) * 100
).round(2)

dataset_unificado['pct_cambio_defunciones'] = (
    (dataset_unificado['dif_defunciones_año_anterior'] / dataset_unificado['defunciones_totales'].shift(1)) * 100
).round(2)

print("✅ Variables derivadas creadas:")
variables_derivadas = [
    'tasa_natalidad', 'tasa_mortalidad', 'ratio_nacimientos_sexo', 'ratio_defunciones_sexo',
    'crecimiento_natural', 'porcentaje_crecimiento_natural', 'dif_nacimientos_año_anterior',
    'dif_defunciones_año_anterior', 'pct_cambio_nacimientos', 'pct_cambio_defunciones'
]

for var in variables_derivadas:
    print(f"  - {var}")

print(f"\n📊 Dataset unificado final: {dataset_unificado.shape[0]} registros, {dataset_unificado.shape[1]} columnas")


=== GENERACIÓN DE VARIABLES DERIVADAS ===
📊 Calculando variables derivadas...
✅ Variables derivadas creadas:
  - tasa_natalidad
  - tasa_mortalidad
  - ratio_nacimientos_sexo
  - ratio_defunciones_sexo
  - crecimiento_natural
  - porcentaje_crecimiento_natural
  - dif_nacimientos_año_anterior
  - dif_defunciones_año_anterior
  - pct_cambio_nacimientos
  - pct_cambio_defunciones

📊 Dataset unificado final: 50 registros, 17 columnas


In [32]:
# 6.4 Integrar información por rangos de edad (CORREGIDO)

print("=== INTEGRACIÓN DE INFORMACIÓN POR RANGOS DE EDAD ===")

# Verificar años comunes entre datasets de edad
años_nacimientos_edad = set(nacimientos_edad_final['año'])
años_defunciones_edad = set(defunciones_edad_final['año'])
años_comunes_edad = años_nacimientos_edad & años_defunciones_edad

print(f"�� Años comunes para integración por edad: {sorted(años_comunes_edad)}")
print(f"�� Total años comunes: {len(años_comunes_edad)}")

# IMPORTANTE: Usar los rangos detallados de defunciones_estandarizado
print(f"\n�� Usando rangos detallados de defunciones_estandarizado...")

# Crear dataset de defunciones por edad con rangos detallados
defunciones_edad_detalladas = defunciones_estandarizado.groupby('año')['rango_edad'].value_counts().unstack(fill_value=0)

# Renombrar columnas para consistencia
defunciones_edad_detalladas.columns = [f'defunciones_{col}' for col in defunciones_edad_detalladas.columns]

# Resetear índice para tener 'año' como columna
defunciones_edad_detalladas = defunciones_edad_detalladas.reset_index()

print(f"✅ Dataset defunciones con rangos detallados: {defunciones_edad_detalladas.shape}")
print(f"📋 Rangos disponibles: {list(defunciones_edad_detalladas.columns[1:])}")

# Filtrar por años comunes
defunciones_edad_detalladas = defunciones_edad_detalladas[defunciones_edad_detalladas['año'].isin(años_comunes_edad)]

# Integrar datasets
dataset_extendido = nacimientos_edad_final.merge(
    defunciones_edad_detalladas, 
    on='año', 
    how='inner'
)

print(f"\n✅ Dataset extendido con rangos detallados: {dataset_extendido.shape}")
print(f"📅 Años incluidos: {sorted(dataset_extendido['año'].tolist())}")

# Mostrar columnas del dataset extendido
print(f"\n📋 Columnas del dataset extendido:")
columnas_edad = [col for col in dataset_extendido.columns if 'edad' in col.lower() or 'nacimientos_' in col or 'defunciones_' in col]
for i, col in enumerate(columnas_edad, 1):
    print(f"  {i:2d}. {col}")

print(f"\n✅ Integración por rangos de edad detallados completada")


=== INTEGRACIÓN DE INFORMACIÓN POR RANGOS DE EDAD ===
�� Años comunes para integración por edad: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
�� Total años comunes: 14

�� Usando rangos detallados de defunciones_estandarizado...
✅ Dataset defunciones con rangos detallados: (11, 23)
📋 Rangos disponibles: ['defunciones_100_mas', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_1_4', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 'defunciones_45_49', 'defunciones_50_54', 'defunciones_55_59', 'defunciones_5_9', 'defunciones_60_64', 'defunciones_65_69', 'defunciones_70_74', 'defunciones_75_79', 'defunciones_80_84', 'defunciones_85_89', 'defunciones_90_94', 'defunciones_95_99', 'defunciones_menores_1']

✅ Dataset extendido con rangos detallados: (10, 32)
📅 Años incluidos: [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

📋 Columnas del dataset extendido:
   1. nacimientos_menor

In [33]:
# 6.5 Generar variables derivadas adicionales por edad (ACTUALIZADO)

print("=== VARIABLES DERIVADAS ADICIONALES POR EDAD ===")

# Calcular totales de nacimientos y defunciones
print("📊 Calculando totales...")

# Calcular total de nacimientos (suma de todos los rangos)
columnas_nacimientos = [col for col in dataset_extendido.columns if col.startswith('nacimientos_')]
dataset_extendido['total_nacimientos'] = dataset_extendido[columnas_nacimientos].sum(axis=1)

# Calcular total de defunciones (suma de todos los rangos)
columnas_defunciones = [col for col in dataset_extendido.columns if col.startswith('defunciones_')]
dataset_extendido['total_defunciones'] = dataset_extendido[columnas_defunciones].sum(axis=1)

print(f"✅ Total nacimientos: {dataset_extendido['total_nacimientos'].sum():,}")
print(f"✅ Total defunciones: {dataset_extendido['total_defunciones'].sum():,}")

# Calcular porcentajes y ratios (igual que antes)
# ... resto del código igual ...

# Concentración de defunciones en adultos mayores (65+ años) - AHORA SÍ FUNCIONA
columnas_65_mas = [col for col in columnas_defunciones if any(rango in col for rango in ['65_69', '70_74', '75_79', '80_84', '85_89', '90_94', '95_99', '100_mas'])]

if columnas_65_mas:
    dataset_extendido['concentracion_defunciones_65_mas'] = dataset_extendido[columnas_65_mas].sum(axis=1) / dataset_extendido['total_defunciones'] * 100
    print(f"✅ Concentración defunciones 65+ años calculada usando {len(columnas_65_mas)} rangos")

=== VARIABLES DERIVADAS ADICIONALES POR EDAD ===
📊 Calculando totales...
✅ Total nacimientos: 2,110,436
✅ Total defunciones: 1,151,279
✅ Concentración defunciones 65+ años calculada usando 8 rangos


In [35]:
# 6.6 Guardar datasets integrados

print("=== GUARDANDO DATASETS INTEGRADOS ===")

# Guardar dataset unificado básico
ruta_unificado = r"C:\ProyectoML2\proyecto-ml\data\02_intermediate\dataset_unificado_temporal.csv"
dataset_unificado.to_csv(ruta_unificado, index=False)
print(f"✅ Dataset unificado guardado: {ruta_unificado}")
print(f"📊 Dimensiones: {dataset_unificado.shape}")

# Guardar dataset extendido con información por edad
ruta_extendido = r"C:\ProyectoML2\proyecto-ml\data\02_intermediate\dataset_extendido_con_edad.csv"
dataset_extendido.to_csv(ruta_extendido, index=False)
print(f"✅ Dataset extendido guardado: {ruta_extendido}")
print(f"📊 Dimensiones: {dataset_extendido.shape}")

# Verificar archivos guardados
if os.path.exists(ruta_unificado) and os.path.exists(ruta_extendido):
    print("\n✅ Verificación: Ambos archivos guardados correctamente")
    
    # Mostrar muestra del dataset unificado
    print("\n📋 Muestra del dataset unificado:")
    muestra_unificado = pd.read_csv(ruta_unificado, nrows=5)
    columnas_unificado = ['año', 'nacimientos_totales', 'defunciones_totales', 'ratio_nacimientos_sexo', 'crecimiento_natural']
    # Verificar que las columnas existen
    columnas_disponibles_unificado = [col for col in columnas_unificado if col in muestra_unificado.columns]
    if columnas_disponibles_unificado:
        display(muestra_unificado[columnas_disponibles_unificado])
    else:
        print("Columnas disponibles:", muestra_unificado.columns.tolist())
    
    # Mostrar muestra del dataset extendido
    print("\n📋 Muestra del dataset extendido:")
    muestra_extendido = pd.read_csv(ruta_extendido, nrows=3)
    
    # Mostrar todas las columnas disponibles primero
    print("Columnas disponibles en dataset extendido:")
    print(muestra_extendido.columns.tolist())
    
    # Seleccionar columnas que existan para mostrar
    columnas_candidatas = ['año', 'total_nacimientos', 'total_defunciones', 'nacimientos_25_29', 'defunciones_50_mas', 'concentracion_nacimientos_25_34']
    columnas_disponibles_extendido = [col for col in columnas_candidatas if col in muestra_extendido.columns]
    
    if columnas_disponibles_extendido:
        print(f"\nMostrando columnas disponibles: {columnas_disponibles_extendido}")
        display(muestra_extendido[columnas_disponibles_extendido])
    else:
        # Mostrar primeras columnas disponibles
        print("\nMostrando primeras columnas disponibles:")
        display(muestra_extendido.iloc[:, :6])  # Primeras 6 columnas
    
    # Mostrar estadísticas básicas
    print(f"\n📊 Estadísticas de datasets guardados:")
    print(f"  Dataset unificado: {muestra_unificado.shape[0]} filas, {muestra_unificado.shape[1]} columnas")
    print(f"  Dataset extendido: {muestra_extendido.shape[0]} filas, {muestra_extendido.shape[1]} columnas")
    
    # Mostrar años incluidos
    if 'año' in muestra_extendido.columns:
        print(f"  Años en dataset extendido: {sorted(muestra_extendido['año'].tolist())}")
    
else:
    print("❌ Error: No se pudieron guardar los archivos")

=== GUARDANDO DATASETS INTEGRADOS ===
✅ Dataset unificado guardado: C:\ProyectoML2\proyecto-ml\data\02_intermediate\dataset_unificado_temporal.csv
📊 Dimensiones: (50, 17)
✅ Dataset extendido guardado: C:\ProyectoML2\proyecto-ml\data\02_intermediate\dataset_extendido_con_edad.csv
📊 Dimensiones: (10, 35)

✅ Verificación: Ambos archivos guardados correctamente

📋 Muestra del dataset unificado:


,año,nacimientos_totales,defunciones_totales,ratio_nacimientos_sexo,crecimiento_natural
0,2023,171992,121270,1.041,50722
1,2022,189310,136958,1.029,52352
2,2021,177255,137439,1.040,39816
3,2020,194952,125833,1.051,69119
4,2019,210188,109658,1.044,100530



📋 Muestra del dataset extendido:
Columnas disponibles en dataset extendido:
['año', 'nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34', 'nacimientos_35_39', 'nacimientos_40_44', 'nacimientos_45_49', 'nacimientos_50_mas', 'defunciones_100_mas', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_1_4', 'defunciones_20_24', 'defunciones_25_29', 'defunciones_30_34', 'defunciones_35_39', 'defunciones_40_44', 'defunciones_45_49', 'defunciones_50_54', 'defunciones_55_59', 'defunciones_5_9', 'defunciones_60_64', 'defunciones_65_69', 'defunciones_70_74', 'defunciones_75_79', 'defunciones_80_84', 'defunciones_85_89', 'defunciones_90_94', 'defunciones_95_99', 'defunciones_menores_1', 'total_nacimientos', 'total_defunciones', 'concentracion_defunciones_65_mas']

Mostrando columnas disponibles: ['año', 'total_nacimientos', 'total_defunciones', 'nacimientos_25_29']


,año,total_nacimientos,total_defunciones,nacimientos_25_29
0,2023,171715,121646,45178
1,2022,189023,136467,50637
2,2021,177067,137208,47769



📊 Estadísticas de datasets guardados:
  Dataset unificado: 5 filas, 17 columnas
  Dataset extendido: 3 filas, 35 columnas
  Años en dataset extendido: [2021, 2022, 2023]


In [37]:
# 6.7 Resumen de la integración de datasets

print("=== RESUMEN DE LA INTEGRACIÓN DE DATASETS ===")

# Resumen de datasets creados
datasets_integrados = {
    "dataset_unificado": dataset_unificado,
    "dataset_extendido": dataset_extendido
}

print("📋 Datasets integrados creados:")
for nombre_dataset, df in datasets_integrados.items():
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Dimensiones: {df.shape}")
    print(f"   Años cubiertos: {df['año'].min()} - {df['año'].max()}")
    print(f"   Total años: {len(df['año'].unique())}")

# Resumen de variables derivadas
print(f"\n📊 Variables derivadas creadas:")
variables_temporales = [col for col in dataset_unificado.columns if col.startswith(('tasa_', 'ratio_', 'crecimiento_', 'dif_', 'pct_cambio_'))]
variables_edad = [col for col in dataset_extendido.columns if col.startswith(('pct_', 'ratio_', 'concentracion_', 'total_'))]

print(f"  Variables temporales: {len(variables_temporales)}")
print(f"  Variables por edad: {len(variables_edad)}")
print(f"  Total variables derivadas: {len(variables_temporales) + len(variables_edad)}")

# Resumen de rangos de edad
print(f"\n📋 Rangos de edad implementados:")
rangos_defunciones = [col for col in dataset_extendido.columns if col.startswith('defunciones_') and not col.startswith('defunciones_totales')]
rangos_nacimientos = [col for col in dataset_extendido.columns if col.startswith('nacimientos_') and not col.startswith('nacimientos_totales')]

print(f"  Rangos de nacimientos: {len(rangos_nacimientos)}")
print(f"  Rangos de defunciones: {len(rangos_defunciones)}")
print(f"  Rangos de defunciones detallados: desde {rangos_defunciones[0]} hasta {rangos_defunciones[-1]}")

# Mostrar algunos rangos específicos
print(f"\n📋 Ejemplos de rangos detallados:")
print(f"  Nacimientos: {rangos_nacimientos[:5]}...")
print(f"  Defunciones: {rangos_defunciones[:5]}...")

# Calcular rangos de adultos mayores por separado
rangos_adultos_mayores = []
for r in rangos_defunciones:
    if any(x in r for x in ['65_69', '70_74', '75_79', '80_84', '85_89', '90_94', '95_99', '100_mas']):
        rangos_adultos_mayores.append(r)

print(f"  Defunciones adultos mayores: {rangos_adultos_mayores}")

print(f"\n=== BENEFICIOS DE LA INTEGRACIÓN ===")
print("✅ Dataset unificado temporal (1974-2023)")
print("✅ Dataset extendido con información por edad (años limitados)")
print("✅ Variables derivadas para análisis demográfico")
print("✅ Tasas de natalidad y mortalidad calculadas")
print("✅ Ratios por sexo y edad")
print("✅ Indicadores de crecimiento natural")
print("✅ Concentración de eventos por edad")
print("✅ Rangos de defunciones detallados (hasta 100+ años)")
print("✅ Totales calculados automáticamente")

print(f"\n=== NOTAS IMPORTANTES ===")
print("📝 Dataset extendido tiene años limitados debido a disponibilidad de datos por edad")
print("📝 Rangos de defunciones incluyen categorías detalladas hasta 100+ años")
print("📝 Variables de concentración calculadas para análisis demográfico")

print(f"\n✅ Integración de datasets completada exitosamente")

=== RESUMEN DE LA INTEGRACIÓN DE DATASETS ===
📋 Datasets integrados creados:

DATASET_UNIFICADO:
   Dimensiones: (50, 17)
   Años cubiertos: 1974 - 2023
   Total años: 50

DATASET_EXTENDIDO:
   Dimensiones: (10, 35)
   Años cubiertos: 2014 - 2023
   Total años: 10

📊 Variables derivadas creadas:
  Variables temporales: 9
  Variables por edad: 3
  Total variables derivadas: 12

📋 Rangos de edad implementados:
  Rangos de nacimientos: 9
  Rangos de defunciones: 22
  Rangos de defunciones detallados: desde defunciones_100_mas hasta defunciones_menores_1

📋 Ejemplos de rangos detallados:
  Nacimientos: ['nacimientos_menores_15', 'nacimientos_15_19', 'nacimientos_20_24', 'nacimientos_25_29', 'nacimientos_30_34']...
  Defunciones: ['defunciones_100_mas', 'defunciones_10_14', 'defunciones_15_19', 'defunciones_1_4', 'defunciones_20_24']...
  Defunciones adultos mayores: ['defunciones_100_mas', 'defunciones_65_69', 'defunciones_70_74', 'defunciones_75_79', 'defunciones_80_84', 'defunciones_85_8

## 7. Validación de Consistencia

Esta sección se enfoca en verificar la consistencia entre datasets y detectar valores anómalos que puedan indicar errores de captura.


In [39]:
# 7.1 Verificar suma de nacimientos por sexo vs totales

print("=== VERIFICACIÓN DE SUMA DE NACIMIENTOS POR SEXO ===")

# Verificar que la suma de nacimientos por sexo coincida con los totales
print("🔍 Verificando consistencia en nacimientos...")

# Para años con información completa (2015-2023)
años_completos = dataset_unificado[dataset_unificado['nacimientos_hombres'].notna()]['año'].tolist()

inconsistencias_nacimientos = []

for año in años_completos:
    fila = dataset_unificado[dataset_unificado['año'] == año].iloc[0]
    
    # Calcular suma por sexo
    suma_por_sexo = fila['nacimientos_hombres'] + fila['nacimientos_mujeres']
    total_registrado = fila['nacimientos_totales']
    
    # Verificar diferencia
    diferencia = abs(suma_por_sexo - total_registrado)
    porcentaje_diferencia = (diferencia / total_registrado) * 100
    
    inconsistencias_nacimientos.append({
        'año': año,
        'suma_por_sexo': suma_por_sexo,
        'total_registrado': total_registrado,
        'diferencia': diferencia,
        'porcentaje': porcentaje_diferencia
    })

print(f"📊 Años verificados: {len(años_completos)}")

# Clasificar inconsistencias
inconsistencias_significativas = [inc for inc in inconsistencias_nacimientos if inc['porcentaje'] > 0.1]
inconsistencias_menores = [inc for inc in inconsistencias_nacimientos if inc['porcentaje'] <= 0.1]

print(f"📊 Inconsistencias menores (<0.1%): {len(inconsistencias_menores)}")
print(f"�� Inconsistencias significativas (>0.1%): {len(inconsistencias_significativas)}")

# Mostrar resumen de inconsistencias menores
if inconsistencias_menores:
    print(f"\n✅ Inconsistencias menores (aceptables):")
    for inc in inconsistencias_menores:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar inconsistencias significativas si las hay
if inconsistencias_significativas:
    print(f"\n⚠️ Inconsistencias significativas (revisar):")
    for inc in inconsistencias_significativas:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar ejemplo de verificación
print(f"\n�� Ejemplo de verificación (año 2023):")
ejemplo_2023 = dataset_unificado[dataset_unificado['año'] == 2023].iloc[0]
suma_ejemplo = ejemplo_2023['nacimientos_hombres'] + ejemplo_2023['nacimientos_mujeres']
diferencia_ejemplo = abs(suma_ejemplo - ejemplo_2023['nacimientos_totales'])
porcentaje_ejemplo = (diferencia_ejemplo / ejemplo_2023['nacimientos_totales']) * 100

print(f"  Nacimientos hombres: {ejemplo_2023['nacimientos_hombres']:,}")
print(f"  Nacimientos mujeres: {ejemplo_2023['nacimientos_mujeres']:,}")
print(f"  Suma por sexo: {suma_ejemplo:,}")
print(f"  Total registrado: {ejemplo_2023['nacimientos_totales']:,}")
print(f"  Diferencia: {diferencia_ejemplo:,} ({porcentaje_ejemplo:.3f}%)")

# Conclusión
print(f"\n📝 CONCLUSIÓN:")
if len(inconsistencias_significativas) == 0:
    print("✅ Todas las inconsistencias son menores y aceptables")
    print("✅ Los datos son consistentes para análisis")
else:
    print("⚠️ Hay inconsistencias significativas que requieren revisión")


=== VERIFICACIÓN DE SUMA DE NACIMIENTOS POR SEXO ===
🔍 Verificando consistencia en nacimientos...
📊 Años verificados: 9
📊 Inconsistencias menores (<0.1%): 9
�� Inconsistencias significativas (>0.1%): 0

✅ Inconsistencias menores (aceptables):
  Año 2023: Diferencia = 17.0 (0.010%)
  Año 2022: Diferencia = 15.0 (0.008%)
  Año 2021: Diferencia = 17.0 (0.010%)
  Año 2020: Diferencia = 19.0 (0.010%)
  Año 2019: Diferencia = 23.0 (0.011%)
  Año 2018: Diferencia = 24.0 (0.011%)
  Año 2017: Diferencia = 25.0 (0.011%)
  Año 2016: Diferencia = 28.0 (0.012%)
  Año 2015: Diferencia = 21.0 (0.009%)

�� Ejemplo de verificación (año 2023):
  Nacimientos hombres: 87,713.0
  Nacimientos mujeres: 84,262.0
  Suma por sexo: 171,975.0
  Total registrado: 171,992.0
  Diferencia: 17.0 (0.010%)

📝 CONCLUSIÓN:
✅ Todas las inconsistencias son menores y aceptables
✅ Los datos son consistentes para análisis


In [41]:
# 7.2 Verificar suma de defunciones por sexo vs totales

print("=== VERIFICACIÓN DE SUMA DE DEFUNCIONES POR SEXO ===")

# Verificar que la suma de defunciones por sexo coincida con los totales
print("🔍 Verificando consistencia en defunciones...")

inconsistencias_defunciones = []

for año in años_completos:
    fila = dataset_unificado[dataset_unificado['año'] == año].iloc[0]
    
    # Calcular suma por sexo
    suma_por_sexo = fila['defunciones_hombres'] + fila['defunciones_mujeres']
    total_registrado = fila['defunciones_totales']
    
    # Verificar diferencia
    diferencia = abs(suma_por_sexo - total_registrado)
    porcentaje_diferencia = (diferencia / total_registrado) * 100
    
    inconsistencias_defunciones.append({
        'año': año,
        'suma_por_sexo': suma_por_sexo,
        'total_registrado': total_registrado,
        'diferencia': diferencia,
        'porcentaje': porcentaje_diferencia
    })

print(f"📊 Años verificados: {len(años_completos)}")

# Clasificar inconsistencias
inconsistencias_significativas = [inc for inc in inconsistencias_defunciones if inc['porcentaje'] > 0.1]
inconsistencias_menores = [inc for inc in inconsistencias_defunciones if inc['porcentaje'] <= 0.1]

print(f"📊 Inconsistencias menores (<0.1%): {len(inconsistencias_menores)}")
print(f"�� Inconsistencias significativas (>0.1%): {len(inconsistencias_significativas)}")

# Mostrar resumen de inconsistencias menores
if inconsistencias_menores:
    print(f"\n✅ Inconsistencias menores (aceptables):")
    for inc in inconsistencias_menores:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar inconsistencias significativas si las hay
if inconsistencias_significativas:
    print(f"\n⚠️ Inconsistencias significativas (revisar):")
    for inc in inconsistencias_significativas:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar ejemplo de verificación
print(f"\n�� Ejemplo de verificación (año 2023):")
ejemplo_2023 = dataset_unificado[dataset_unificado['año'] == 2023].iloc[0]
suma_ejemplo = ejemplo_2023['defunciones_hombres'] + ejemplo_2023['defunciones_mujeres']
diferencia_ejemplo = abs(suma_ejemplo - ejemplo_2023['defunciones_totales'])
porcentaje_ejemplo = (diferencia_ejemplo / ejemplo_2023['defunciones_totales']) * 100

print(f"  Defunciones hombres: {ejemplo_2023['defunciones_hombres']:,}")
print(f"  Defunciones mujeres: {ejemplo_2023['defunciones_mujeres']:,}")
print(f"  Suma por sexo: {suma_ejemplo:,}")
print(f"  Total registrado: {ejemplo_2023['defunciones_totales']:,}")
print(f"  Diferencia: {diferencia_ejemplo:,} ({porcentaje_ejemplo:.3f}%)")

# Conclusión
print(f"\n📝 CONCLUSIÓN:")
if len(inconsistencias_significativas) == 0:
    print("✅ Todas las inconsistencias son menores y aceptables")
    print("✅ Los datos de defunciones son consistentes para análisis")
else:
    print("⚠️ Hay inconsistencias significativas que requieren revisión")


=== VERIFICACIÓN DE SUMA DE DEFUNCIONES POR SEXO ===
🔍 Verificando consistencia en defunciones...
📊 Años verificados: 9
📊 Inconsistencias menores (<0.1%): 9
�� Inconsistencias significativas (>0.1%): 0

✅ Inconsistencias menores (aceptables):
  Año 2023: Diferencia = 11.0 (0.009%)
  Año 2022: Diferencia = 7.0 (0.005%)
  Año 2021: Diferencia = 12.0 (0.009%)
  Año 2020: Diferencia = 13.0 (0.010%)
  Año 2019: Diferencia = 16.0 (0.015%)
  Año 2018: Diferencia = 19.0 (0.018%)
  Año 2017: Diferencia = 22.0 (0.021%)
  Año 2016: Diferencia = 26.0 (0.025%)
  Año 2015: Diferencia = 19.0 (0.018%)

�� Ejemplo de verificación (año 2023):
  Defunciones hombres: 63,174.0
  Defunciones mujeres: 58,085.0
  Suma por sexo: 121,259.0
  Total registrado: 121,270.0
  Diferencia: 11.0 (0.009%)

📝 CONCLUSIÓN:
✅ Todas las inconsistencias son menores y aceptables
✅ Los datos de defunciones son consistentes para análisis


In [42]:
# 7.3 Verificar suma de nacimientos por edad vs totales

print("=== VERIFICACIÓN DE SUMA DE NACIMIENTOS POR EDAD ===")

# Verificar que la suma de nacimientos por edad coincida con los totales
print("🔍 Verificando consistencia en nacimientos por edad...")

# Usar años disponibles en dataset_extendido
años_edad = sorted(dataset_extendido['año'].tolist())
print(f"�� Años disponibles para verificación por edad: {años_edad}")

inconsistencias_nacimientos_edad = []

for año in años_edad:
    fila = dataset_extendido[dataset_extendido['año'] == año].iloc[0]
    
    # Calcular suma por rangos de edad
    columnas_nacimientos_edad = [col for col in fila.index if col.startswith('nacimientos_') and col != 'total_nacimientos']
    suma_por_edad = fila[columnas_nacimientos_edad].sum()
    total_registrado = fila['total_nacimientos']
    
    # Verificar diferencia
    diferencia = abs(suma_por_edad - total_registrado)
    porcentaje_diferencia = (diferencia / total_registrado) * 100
    
    inconsistencias_nacimientos_edad.append({
        'año': año,
        'suma_por_edad': suma_por_edad,
        'total_registrado': total_registrado,
        'diferencia': diferencia,
        'porcentaje': porcentaje_diferencia
    })

print(f"📊 Años verificados: {len(años_edad)}")

# Clasificar inconsistencias
inconsistencias_significativas = [inc for inc in inconsistencias_nacimientos_edad if inc['porcentaje'] > 0.1]
inconsistencias_menores = [inc for inc in inconsistencias_nacimientos_edad if inc['porcentaje'] <= 0.1]

print(f"📊 Inconsistencias menores (<0.1%): {len(inconsistencias_menores)}")
print(f" Inconsistencias significativas (>0.1%): {len(inconsistencias_significativas)}")

# Mostrar resumen de inconsistencias menores
if inconsistencias_menores:
    print(f"\n✅ Inconsistencias menores (aceptables):")
    for inc in inconsistencias_menores:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar inconsistencias significativas si las hay
if inconsistencias_significativas:
    print(f"\n⚠️ Inconsistencias significativas (revisar):")
    for inc in inconsistencias_significativas:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar ejemplo de verificación
print(f"\n Ejemplo de verificación (año 2023):")
ejemplo_2023 = dataset_extendido[dataset_extendido['año'] == 2023].iloc[0]
columnas_nacimientos_edad = [col for col in ejemplo_2023.index if col.startswith('nacimientos_') and col != 'total_nacimientos']
suma_ejemplo = ejemplo_2023[columnas_nacimientos_edad].sum()
diferencia_ejemplo = abs(suma_ejemplo - ejemplo_2023['total_nacimientos'])
porcentaje_ejemplo = (diferencia_ejemplo / ejemplo_2023['total_nacimientos']) * 100

print(f"  Suma por rangos de edad: {suma_ejemplo:,}")
print(f"  Total registrado: {ejemplo_2023['total_nacimientos']:,}")
print(f"  Diferencia: {diferencia_ejemplo:,} ({porcentaje_ejemplo:.3f}%)")

# Mostrar rangos de edad incluidos
print(f"\n📋 Rangos de edad incluidos en verificación:")
for i, col in enumerate(columnas_nacimientos_edad, 1):
    print(f"  {i:2d}. {col}")

# Conclusión
print(f"\n📝 CONCLUSIÓN:")
if len(inconsistencias_significativas) == 0:
    print("✅ Todas las inconsistencias son menores y aceptables")
    print("✅ Los datos de nacimientos por edad son consistentes para análisis")
else:
    print("⚠️ Hay inconsistencias significativas que requieren revisión")

=== VERIFICACIÓN DE SUMA DE NACIMIENTOS POR EDAD ===
🔍 Verificando consistencia en nacimientos por edad...
�� Años disponibles para verificación por edad: [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
📊 Años verificados: 10
📊 Inconsistencias menores (<0.1%): 10
 Inconsistencias significativas (>0.1%): 0

✅ Inconsistencias menores (aceptables):
  Año 2014: Diferencia = 0.0 (0.000%)
  Año 2015: Diferencia = 0.0 (0.000%)
  Año 2016: Diferencia = 0.0 (0.000%)
  Año 2017: Diferencia = 0.0 (0.000%)
  Año 2018: Diferencia = 0.0 (0.000%)
  Año 2019: Diferencia = 0.0 (0.000%)
  Año 2020: Diferencia = 0.0 (0.000%)
  Año 2021: Diferencia = 0.0 (0.000%)
  Año 2022: Diferencia = 0.0 (0.000%)
  Año 2023: Diferencia = 0.0 (0.000%)

 Ejemplo de verificación (año 2023):
  Suma por rangos de edad: 171,715.0
  Total registrado: 171,715.0
  Diferencia: 0.0 (0.000%)

📋 Rangos de edad incluidos en verificación:
   1. nacimientos_menores_15
   2. nacimientos_15_19
   3. nacimientos_20_24
   4.

In [44]:
# 7.4 Verificar suma de defunciones por edad vs totales

print("=== VERIFICACIÓN DE SUMA DE DEFUNCIONES POR EDAD ===")

# Verificar que la suma de defunciones por edad coincida con los totales
print("🔍 Verificando consistencia en defunciones por edad...")

# Usar años disponibles en dataset_extendido
años_edad = sorted(dataset_extendido['año'].tolist())
print(f"�� Años disponibles para verificación por edad: {años_edad}")

inconsistencias_defunciones_edad = []

for año in años_edad:
    fila = dataset_extendido[dataset_extendido['año'] == año].iloc[0]
    
    # Calcular suma por rangos de edad
    columnas_defunciones_edad = [col for col in fila.index if col.startswith('defunciones_') and col != 'total_defunciones']
    suma_por_edad = fila[columnas_defunciones_edad].sum()
    total_registrado = fila['total_defunciones']
    
    # Verificar diferencia
    diferencia = abs(suma_por_edad - total_registrado)
    porcentaje_diferencia = (diferencia / total_registrado) * 100
    
    inconsistencias_defunciones_edad.append({
        'año': año,
        'suma_por_edad': suma_por_edad,
        'total_registrado': total_registrado,
        'diferencia': diferencia,
        'porcentaje': porcentaje_diferencia
    })

print(f"📊 Años verificados: {len(años_edad)}")

# Clasificar inconsistencias
inconsistencias_significativas = [inc for inc in inconsistencias_defunciones_edad if inc['porcentaje'] > 0.1]
inconsistencias_menores = [inc for inc in inconsistencias_defunciones_edad if inc['porcentaje'] <= 0.1]

print(f"📊 Inconsistencias menores (<0.1%): {len(inconsistencias_menores)}")
print(f"⚠️ Inconsistencias significativas (>0.1%): {len(inconsistencias_significativas)}")

# Mostrar resumen de inconsistencias menores
if inconsistencias_menores:
    print(f"\n✅ Inconsistencias menores (aceptables):")
    for inc in inconsistencias_menores:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar inconsistencias significativas si las hay
if inconsistencias_significativas:
    print(f"\n⚠️ Inconsistencias significativas (revisar):")
    for inc in inconsistencias_significativas:
        print(f"  Año {inc['año']}: Diferencia = {inc['diferencia']:,} ({inc['porcentaje']:.3f}%)")

# Mostrar ejemplo de verificación
print(f"\n�� Ejemplo de verificación (año 2023):")
ejemplo_2023 = dataset_extendido[dataset_extendido['año'] == 2023].iloc[0]
columnas_defunciones_edad = [col for col in ejemplo_2023.index if col.startswith('defunciones_') and col != 'total_defunciones']
suma_ejemplo = ejemplo_2023[columnas_defunciones_edad].sum()
diferencia_ejemplo = abs(suma_ejemplo - ejemplo_2023['total_defunciones'])
porcentaje_ejemplo = (diferencia_ejemplo / ejemplo_2023['total_defunciones']) * 100

print(f"  Suma por rangos de edad: {suma_ejemplo:,}")
print(f"  Total registrado: {ejemplo_2023['total_defunciones']:,}")
print(f"  Diferencia: {diferencia_ejemplo:,} ({porcentaje_ejemplo:.3f}%)")

# Mostrar rangos de edad incluidos
print(f"\n📋 Rangos de edad incluidos en verificación:")
for i, col in enumerate(columnas_defunciones_edad, 1):
    print(f"  {i:2d}. {col}")

# Mostrar estadísticas de rangos
print(f"\n📊 Estadísticas de rangos de defunciones:")
print(f"  Total rangos verificados: {len(columnas_defunciones_edad)}")
print(f"  Rango más bajo: {min(columnas_defunciones_edad)}")
print(f"  Rango más alto: {max(columnas_defunciones_edad)}")

# Conclusión
print(f"\n📝 CONCLUSIÓN:")
if len(inconsistencias_significativas) == 0:
    print("✅ Todas las inconsistencias son menores y aceptables")
    print("✅ Los datos de defunciones por edad son consistentes para análisis")
    print("✅ Los 22 rangos detallados funcionan correctamente")
else:
    print("⚠️ Hay inconsistencias significativas que requieren revisión")

=== VERIFICACIÓN DE SUMA DE DEFUNCIONES POR EDAD ===
🔍 Verificando consistencia en defunciones por edad...
�� Años disponibles para verificación por edad: [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
📊 Años verificados: 10
📊 Inconsistencias menores (<0.1%): 10
⚠️ Inconsistencias significativas (>0.1%): 0

✅ Inconsistencias menores (aceptables):
  Año 2014: Diferencia = 0.0 (0.000%)
  Año 2015: Diferencia = 0.0 (0.000%)
  Año 2016: Diferencia = 0.0 (0.000%)
  Año 2017: Diferencia = 0.0 (0.000%)
  Año 2018: Diferencia = 0.0 (0.000%)
  Año 2019: Diferencia = 0.0 (0.000%)
  Año 2020: Diferencia = 0.0 (0.000%)
  Año 2021: Diferencia = 0.0 (0.000%)
  Año 2022: Diferencia = 0.0 (0.000%)
  Año 2023: Diferencia = 0.0 (0.000%)

�� Ejemplo de verificación (año 2023):
  Suma por rangos de edad: 121,646.0
  Total registrado: 121,646.0
  Diferencia: 0.0 (0.000%)

📋 Rangos de edad incluidos en verificación:
   1. defunciones_100_mas
   2. defunciones_10_14
   3. defunciones_15_19
   4

In [45]:
# 7.5 Detección de outliers en variables numéricas

print("=== DETECCIÓN DE OUTLIERS ===")

# Función para detectar outliers usando el método IQR
def detectar_outliers_iqr(serie, nombre_variable):
    """
    Detecta outliers usando el método del rango intercuartílico (IQR)
    """
    Q1 = serie.quantile(0.25)
    Q3 = serie.quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = serie[(serie < limite_inferior) | (serie > limite_superior)]
    
    return {
        'variable': nombre_variable,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior,
        'outliers': outliers,
        'cantidad_outliers': len(outliers),
        'porcentaje_outliers': (len(outliers) / len(serie)) * 100
    }

# Variables numéricas principales para análisis de outliers
variables_analisis = [
    'nacimientos_totales', 'defunciones_totales', 'nacimientos_hombres', 'nacimientos_mujeres',
    'defunciones_hombres', 'defunciones_mujeres', 'crecimiento_natural', 'ratio_nacimientos_sexo',
    'ratio_defunciones_sexo', 'pct_cambio_nacimientos', 'pct_cambio_defunciones'
]

print("🔍 Analizando outliers en variables principales...")

resultados_outliers = []

for var in variables_analisis:
    if var in dataset_unificado.columns:
        serie = dataset_unificado[var].dropna()
        resultado = detectar_outliers_iqr(serie, var)
        resultados_outliers.append(resultado)

# Mostrar resultados
print("\n📊 Resumen de outliers detectados:")
for resultado in resultados_outliers:
    print(f"\n{resultado['variable'].upper()}:")
    print(f"  Outliers detectados: {resultado['cantidad_outliers']} ({resultado['porcentaje_outliers']:.1f}%)")
    print(f"  Límites: [{resultado['limite_inferior']:.1f}, {resultado['limite_superior']:.1f}]")
    
    if resultado['cantidad_outliers'] > 0:
        print(f"  Valores outliers: {sorted(resultado['outliers'].tolist())}")

# Identificar años con múltiples outliers (CORREGIDO)
print(f"\n🔍 Años con valores atípicos:")
años_outliers = set()

for resultado in resultados_outliers:
    if resultado['cantidad_outliers'] > 0:
        # CORRECCIÓN: Usar el índice correcto
        for idx in resultado['outliers'].index:
            año_outlier = dataset_unificado.loc[idx, 'año']
            años_outliers.add(año_outlier)

if años_outliers:
    print(f"Años con outliers: {sorted(años_outliers)}")
    
    # Mostrar detalles de años problemáticos (CORREGIDO)
    for año in sorted(años_outliers):
        print(f"\n📋 Año {año} - Valores atípicos:")
        fila = dataset_unificado[dataset_unificado['año'] == año].iloc[0]
        
        for resultado in resultados_outliers:
            if resultado['cantidad_outliers'] > 0:
                # Verificar si este año tiene outliers en esta variable
                año_idx = dataset_unificado[dataset_unificado['año'] == año].index[0]
                if año_idx in resultado['outliers'].index:
                    valor = fila[resultado['variable']]
                    print(f"  {resultado['variable']}: {valor:,} (OUTLIER)")
else:
    print("No se detectaron años con múltiples outliers")

# Análisis adicional: identificar patrones temporales
print(f"\n📊 Análisis de patrones temporales:")
for resultado in resultados_outliers:
    if resultado['cantidad_outliers'] > 0:
        print(f"\n{resultado['variable'].upper()}:")
        # Obtener años de outliers
        años_outliers_var = [dataset_unificado.loc[idx, 'año'] for idx in resultado['outliers'].index]
        print(f"  Años con outliers: {sorted(años_outliers_var)}")
        
        # Analizar si hay patrones (ej: concentración en ciertos años)
        if len(set(años_outliers_var)) <= 3:
            print(f"  ⚠️ Concentración en pocos años - revisar datos de esos períodos")
        else:
            print(f"  ✅ Outliers distribuidos en múltiples años")

=== DETECCIÓN DE OUTLIERS ===
🔍 Analizando outliers en variables principales...

📊 Resumen de outliers detectados:

NACIMIENTOS_TOTALES:
  Outliers detectados: 3 (6.0%)
  Límites: [191990.8, 295932.8]
  Valores outliers: [171992, 177255, 189310]

DEFUNCIONES_TOTALES:
  Outliers detectados: 2 (4.0%)
  Límites: [40617.5, 131139.5]
  Valores outliers: [136958, 137439]

NACIMIENTOS_HOMBRES:
  Outliers detectados: 0 (0.0%)
  Límites: [70469.0, 138581.0]

NACIMIENTOS_MUJERES:
  Outliers detectados: 0 (0.0%)
  Límites: [70208.0, 131744.0]

DEFUNCIONES_HOMBRES:
  Outliers detectados: 0 (0.0%)
  Límites: [38253.0, 84973.0]

DEFUNCIONES_MUJERES:
  Outliers detectados: 0 (0.0%)
  Límites: [38932.0, 70028.0]

CRECIMIENTO_NATURAL:
  Outliers detectados: 4 (8.0%)
  Límites: [89589.0, 236759.0]
  Valores outliers: [39816, 50722, 52352, 69119]

RATIO_NACIMIENTOS_SEXO:
  Outliers detectados: 3 (33.3%)
  Límites: [1.0, 1.0]
  Valores outliers: [1.029, 1.034, 1.051]

RATIO_DEFUNCIONES_SEXO:
  Outliers de

In [46]:
# 7.6 Validación de valores lógicos y rangos

print("=== VALIDACIÓN DE VALORES LÓGICOS Y RANGOS ===")

# Validar valores lógicos en variables clave
print("🔍 Validando valores lógicos...")

# 1. Verificar que nacimientos y defunciones sean positivos
nacimientos_negativos = dataset_unificado[dataset_unificado['nacimientos_totales'] < 0]
defunciones_negativas = dataset_unificado[dataset_unificado['defunciones_totales'] < 0]

print(f"📊 Registros con nacimientos negativos: {len(nacimientos_negativos)}")
print(f"📊 Registros con defunciones negativas: {len(defunciones_negativas)}")

# 2. Verificar ratios de sexo (deben estar entre 0.8 y 1.2 aproximadamente)
ratios_nacimientos_extremos = dataset_unificado[
    (dataset_unificado['ratio_nacimientos_sexo'] < 0.8) | 
    (dataset_unificado['ratio_nacimientos_sexo'] > 1.2)
]

ratios_defunciones_extremos = dataset_unificado[
    (dataset_unificado['ratio_defunciones_sexo'] < 0.8) | 
    (dataset_unificado['ratio_defunciones_sexo'] > 1.2)
]

print(f"�� Registros con ratio nacimientos extremo (<0.8 o >1.2): {len(ratios_nacimientos_extremos)}")
print(f"�� Registros con ratio defunciones extremo (<0.8 o >1.2): {len(ratios_defunciones_extremos)}")

# 3. Verificar cambios porcentuales extremos (>50% cambio año a año)
# CORRECCIÓN: Usar abs() correctamente
cambios_nacimientos_extremos = dataset_unificado[
    (dataset_unificado['pct_cambio_nacimientos'].abs() > 50)
]

cambios_defunciones_extremos = dataset_unificado[
    (dataset_unificado['pct_cambio_defunciones'].abs() > 50)
]

print(f"📊 Registros con cambio nacimientos >50%: {len(cambios_nacimientos_extremos)}")
print(f"📊 Registros con cambio defunciones >50%: {len(cambios_defunciones_extremos)}")

# Mostrar ejemplos de valores problemáticos
if len(ratios_nacimientos_extremos) > 0:
    print(f"\n⚠️ Ejemplos de ratios de nacimientos extremos:")
    display(ratios_nacimientos_extremos[['año', 'nacimientos_hombres', 'nacimientos_mujeres', 'ratio_nacimientos_sexo']].head())

if len(cambios_nacimientos_extremos) > 0:
    print(f"\n⚠️ Ejemplos de cambios extremos en nacimientos:")
    display(cambios_nacimientos_extremos[['año', 'nacimientos_totales', 'dif_nacimientos_año_anterior', 'pct_cambio_nacimientos']].head())

# 4. Verificar consistencia temporal (no debe haber saltos imposibles)
print(f"\n🔍 Verificando consistencia temporal...")

# CORRECCIÓN: Calcular diferencias año a año correctamente
dataset_unificado['dif_nacimientos_abs'] = dataset_unificado['nacimientos_totales'].diff().abs()
dataset_unificado['dif_defunciones_abs'] = dataset_unificado['defunciones_totales'].diff().abs()

# Identificar cambios muy grandes (más de 50,000 en un año)
cambios_grandes_nacimientos = dataset_unificado[dataset_unificado['dif_nacimientos_abs'] > 50000]
cambios_grandes_defunciones = dataset_unificado[dataset_unificado['dif_defunciones_abs'] > 50000]

print(f"📊 Años con cambios grandes en nacimientos (>50,000): {len(cambios_grandes_nacimientos)}")
print(f"📊 Años con cambios grandes en defunciones (>50,000): {len(cambios_grandes_defunciones)}")

if len(cambios_grandes_nacimientos) > 0:
    print(f"\n⚠️ Años con cambios grandes en nacimientos:")
    display(cambios_grandes_nacimientos[['año', 'nacimientos_totales', 'dif_nacimientos_año_anterior', 'dif_nacimientos_abs']])

if len(cambios_grandes_defunciones) > 0:
    print(f"\n⚠️ Años con cambios grandes en defunciones:")
    display(cambios_grandes_defunciones[['año', 'defunciones_totales', 'dif_defunciones_año_anterior', 'dif_defunciones_abs']])

# 5. Verificación adicional: rangos de valores esperados
print(f"\n🔍 Verificando rangos de valores esperados...")

# Verificar que los totales sean consistentes con la suma por sexo
dataset_unificado['suma_nacimientos_sexo'] = dataset_unificado['nacimientos_hombres'] + dataset_unificado['nacimientos_mujeres']
dataset_unificado['suma_defunciones_sexo'] = dataset_unificado['defunciones_hombres'] + dataset_unificado['defunciones_mujeres']

# Calcular diferencias
dataset_unificado['diff_nacimientos'] = abs(dataset_unificado['nacimientos_totales'] - dataset_unificado['suma_nacimientos_sexo'])
dataset_unificado['diff_defunciones'] = abs(dataset_unificado['defunciones_totales'] - dataset_unificado['suma_defunciones_sexo'])

# Identificar inconsistencias significativas
inconsistencias_nacimientos = dataset_unificado[dataset_unificado['diff_nacimientos'] > 100]
inconsistencias_defunciones = dataset_unificado[dataset_unificado['diff_defunciones'] > 100]

print(f"📊 Años con inconsistencias en nacimientos (>100 diferencia): {len(inconsistencias_nacimientos)}")
print(f"�� Años con inconsistencias en defunciones (>100 diferencia): {len(inconsistencias_defunciones)}")

if len(inconsistencias_nacimientos) > 0:
    print(f"\n⚠️ Años con inconsistencias en nacimientos:")
    display(inconsistencias_nacimientos[['año', 'nacimientos_totales', 'suma_nacimientos_sexo', 'diff_nacimientos']])

if len(inconsistencias_defunciones) > 0:
    print(f"\n⚠️ Años con inconsistencias en defunciones:")
    display(inconsistencias_defunciones[['año', 'defunciones_totales', 'suma_defunciones_sexo', 'diff_defunciones']])

# Resumen final
print(f"\n📝 RESUMEN DE VALIDACIÓN:")
print(f"✅ Nacimientos negativos: {len(nacimientos_negativos)}")
print(f"✅ Defunciones negativas: {len(defunciones_negativas)}")
print(f"✅ Ratios extremos nacimientos: {len(ratios_nacimientos_extremos)}")
print(f"✅ Ratios extremos defunciones: {len(ratios_defunciones_extremos)}")
print(f"✅ Cambios extremos nacimientos: {len(cambios_nacimientos_extremos)}")
print(f"✅ Cambios extremos defunciones: {len(cambios_defunciones_extremos)}")
print(f"✅ Cambios grandes temporales: {len(cambios_grandes_nacimientos) + len(cambios_grandes_defunciones)}")
print(f"✅ Inconsistencias internas: {len(inconsistencias_nacimientos) + len(inconsistencias_defunciones)}")

if (len(nacimientos_negativos) == 0 and len(defunciones_negativas) == 0 and 
    len(ratios_nacimientos_extremos) == 0 and len(ratios_defunciones_extremos) == 0 and
    len(cambios_nacimientos_extremos) == 0 and len(cambios_defunciones_extremos) == 0 and
    len(cambios_grandes_nacimientos) == 0 and len(cambios_grandes_defunciones) == 0 and
    len(inconsistencias_nacimientos) == 0 and len(inconsistencias_defunciones) == 0):
    print(f"\n🎉 ¡TODOS LOS VALORES SON LÓGICOS Y CONSISTENTES!")
else:
    print(f"\n⚠️ Se encontraron algunos valores que requieren revisión")


=== VALIDACIÓN DE VALORES LÓGICOS Y RANGOS ===
🔍 Validando valores lógicos...
📊 Registros con nacimientos negativos: 0
📊 Registros con defunciones negativas: 0
�� Registros con ratio nacimientos extremo (<0.8 o >1.2): 0
�� Registros con ratio defunciones extremo (<0.8 o >1.2): 0
📊 Registros con cambio nacimientos >50%: 0
📊 Registros con cambio defunciones >50%: 0

🔍 Verificando consistencia temporal...
📊 Años con cambios grandes en nacimientos (>50,000): 0
📊 Años con cambios grandes en defunciones (>50,000): 0

🔍 Verificando rangos de valores esperados...
📊 Años con inconsistencias en nacimientos (>100 diferencia): 0
�� Años con inconsistencias en defunciones (>100 diferencia): 0

📝 RESUMEN DE VALIDACIÓN:
✅ Nacimientos negativos: 0
✅ Defunciones negativas: 0
✅ Ratios extremos nacimientos: 0
✅ Ratios extremos defunciones: 0
✅ Cambios extremos nacimientos: 0
✅ Cambios extremos defunciones: 0
✅ Cambios grandes temporales: 0
✅ Inconsistencias internas: 0

🎉 ¡TODOS LOS VALORES SON LÓGICOS Y

In [47]:
# 7.7 Resumen de la validación de consistencia

print("=== RESUMEN DE LA VALIDACIÓN DE CONSISTENCIA ===")

# Verificar que las variables necesarias estén definidas
try:
    años_con_edad = sorted(dataset_extendido['año'].tolist())
except:
    años_con_edad = []

# Crear resumen de todas las validaciones realizadas
resumen_validaciones = {
    "Verificaciones realizadas": [
        "Suma de nacimientos por sexo vs totales",
        "Suma de defunciones por sexo vs totales", 
        "Suma de nacimientos por edad vs totales",
        "Suma de defunciones por edad vs totales",
        "Detección de outliers en variables numéricas",
        "Validación de valores lógicos y rangos",
        "Verificación de consistencia temporal"
    ],
    "Inconsistencias encontradas": {
        "nacimientos_por_sexo": len(inconsistencias_nacimientos),
        "defunciones_por_sexo": len(inconsistencias_defunciones),
        "nacimientos_por_edad": len(inconsistencias_nacimientos_edad),
        "defunciones_por_edad": len(inconsistencias_defunciones_edad)
    },
    "Outliers detectados": {
        "total_variables_analizadas": len(resultados_outliers),
        "variables_con_outliers": len([r for r in resultados_outliers if r['cantidad_outliers'] > 0]),
        "años_con_outliers": len(años_outliers)
    },
    "Valores problemáticos": {
        "nacimientos_negativos": len(nacimientos_negativos),
        "defunciones_negativas": len(defunciones_negativas),
        "ratios_extremos_nacimientos": len(ratios_nacimientos_extremos),
        "ratios_extremos_defunciones": len(ratios_defunciones_extremos),
        "cambios_extremos_nacimientos": len(cambios_nacimientos_extremos),
        "cambios_extremos_defunciones": len(cambios_defunciones_extremos)
    }
}

print("📋 Resumen de validaciones:")
for categoria, items in resumen_validaciones.items():
    print(f"\n{categoria.upper()}:")
    if isinstance(items, list):
        for item in items:
            print(f"  ✅ {item}")
    else:
        for clave, valor in items.items():
            print(f"  {clave}: {valor}")

# Calcular puntuación de calidad de datos (CORREGIDO)
total_verificaciones = 0
total_problemas = 0

# Contar problemas encontrados
total_problemas += len(inconsistencias_nacimientos)
total_problemas += len(inconsistencias_defunciones)
total_problemas += len(inconsistencias_nacimientos_edad)
total_problemas += len(inconsistencias_defunciones_edad)
total_problemas += len(nacimientos_negativos)
total_problemas += len(defunciones_negativas)
total_problemas += len(ratios_nacimientos_extremos)
total_problemas += len(ratios_defunciones_extremos)

# CORRECCIÓN: Contar verificaciones realizadas correctamente
total_verificaciones += len(años_completos) * 2  # nacimientos y defunciones por sexo
total_verificaciones += len(años_con_edad) * 2  # nacimientos y defunciones por edad
total_verificaciones += len(dataset_unificado) * 4  # valores negativos y ratios extremos

# Agregar verificaciones de outliers
total_verificaciones += len(resultados_outliers) * len(dataset_unificado)

puntuacion_calidad = ((total_verificaciones - total_problemas) / total_verificaciones) * 100

print(f"\n📊 PUNTUACIÓN DE CALIDAD DE DATOS:")
print(f"Verificaciones realizadas: {total_verificaciones:,}")
print(f"Problemas encontrados: {total_problemas:,}")
print(f"Puntuación de calidad: {puntuacion_calidad:.1f}%")

if puntuacion_calidad >= 95:
    print("✅ CALIDAD EXCELENTE: Los datos están muy bien estructurados")
elif puntuacion_calidad >= 90:
    print("✅ CALIDAD BUENA: Los datos tienen calidad aceptable")
elif puntuacion_calidad >= 80:
    print("⚠️ CALIDAD REGULAR: Se recomienda revisar algunos valores")
else:
    print("❌ CALIDAD BAJA: Se requiere limpieza adicional")

# Resumen final de la sección 7
print(f"\n=== RESUMEN FINAL DE LA SECCIÓN 7 ===")
print(f"✅ Validación de consistencia completada exitosamente")
print(f"✅ Todos los datasets están listos para modelado")
print(f"✅ Calidad de datos verificada en múltiples dimensiones")
print(f"✅ Base sólida para análisis y machine learning")

print(f"\n✅ Validación de consistencia completada")


=== RESUMEN DE LA VALIDACIÓN DE CONSISTENCIA ===
📋 Resumen de validaciones:

VERIFICACIONES REALIZADAS:
  ✅ Suma de nacimientos por sexo vs totales
  ✅ Suma de defunciones por sexo vs totales
  ✅ Suma de nacimientos por edad vs totales
  ✅ Suma de defunciones por edad vs totales
  ✅ Detección de outliers en variables numéricas
  ✅ Validación de valores lógicos y rangos
  ✅ Verificación de consistencia temporal

INCONSISTENCIAS ENCONTRADAS:
  nacimientos_por_sexo: 0
  defunciones_por_sexo: 0
  nacimientos_por_edad: 10
  defunciones_por_edad: 10

OUTLIERS DETECTADOS:
  total_variables_analizadas: 11
  variables_con_outliers: 6
  años_con_outliers: 7

VALORES PROBLEMÁTICOS:
  nacimientos_negativos: 0
  defunciones_negativas: 0
  ratios_extremos_nacimientos: 0
  ratios_extremos_defunciones: 0
  cambios_extremos_nacimientos: 0
  cambios_extremos_defunciones: 0

📊 PUNTUACIÓN DE CALIDAD DE DATOS:
Verificaciones realizadas: 788
Problemas encontrados: 20
Puntuación de calidad: 97.5%
✅ CALIDAD E

## 8. Preparación para Modelado

Esta sección se enfoca en crear variables y features necesarios para modelos de Machine Learning, incluyendo codificaciones categóricas, features temporales y variables de tendencia.


In [48]:
# 8.1 Crear variables categóricas para el dataset de defunciones detalladas

print("=== CREACIÓN DE VARIABLES CATEGÓRICAS ===")

# Crear dataset preparado para modelado basado en defunciones detalladas
dataset_modelado = defunciones_estandarizado.copy()
print(f"📊 Dataset base para modelado: {dataset_modelado.shape}")

# Verificar columnas disponibles
print(f"\n📋 Columnas disponibles en dataset_modelado:")
print(dataset_modelado.columns.tolist())

# 1. Codificación de regiones (Label Encoding)
print("\n🔢 Codificando regiones...")
if 'region' in dataset_modelado.columns:
    regiones_unicas = sorted(dataset_modelado['region'].unique())
    mapeo_regiones = {region: i for i, region in enumerate(regiones_unicas)}
    dataset_modelado['region_codificada'] = dataset_modelado['region'].map(mapeo_regiones)
    
    print(f"Regiones codificadas: {len(regiones_unicas)}")
    for region, codigo in mapeo_regiones.items():
        print(f"  {codigo}: {region}")
else:
    print("⚠️ Columna 'region' no encontrada")

# 2. Codificación de sexo (Binary Encoding)
print("\n🔢 Codificando sexo...")
if 'sexo' in dataset_modelado.columns:
    mapeo_sexo = {'Hombre': 1, 'Mujer': 0}
    dataset_modelado['sexo_codificado'] = dataset_modelado['sexo'].map(mapeo_sexo)
    print(f"Sexo codificado: {mapeo_sexo}")
else:
    print("⚠️ Columna 'sexo' no encontrada")

# 3. Codificación de rangos de edad (ya tenemos la columna 'rango_edad')
print("\n🔢 Codificando rangos de edad...")
if 'rango_edad' in dataset_modelado.columns:
    rangos_edad_unicos = sorted(dataset_modelado['rango_edad'].unique())
    mapeo_rangos_edad = {rango: i for i, rango in enumerate(rangos_edad_unicos)}
    dataset_modelado['rango_edad_codificado'] = dataset_modelado['rango_edad'].map(mapeo_rangos_edad)
    
    print(f"Rangos de edad codificados: {len(rangos_edad_unicos)}")
    for rango, codigo in mapeo_rangos_edad.items():
        print(f"  {codigo}: {rango}")
else:
    print("⚠️ Columna 'rango_edad' no encontrada")

# 4. Codificación de códigos de diagnóstico (agrupar por categorías principales)
print("\n🔢 Codificando códigos de diagnóstico...")

# Verificar si existe columna de diagnóstico
columnas_diagnostico = [col for col in dataset_modelado.columns if 'diagnostico' in col.lower() or 'cie' in col.lower()]
print(f"Columnas relacionadas con diagnóstico: {columnas_diagnostico}")

if columnas_diagnostico:
    columna_diagnostico = columnas_diagnostico[0]  # Usar la primera encontrada
    print(f"Usando columna: {columna_diagnostico}")
    
    # Crear función para categorizar códigos CIE-10
    def categorizar_diagnostico(codigo):
        """
        Categoriza códigos CIE-10 en grupos principales
        """
        if pd.isna(codigo):
            return 'desconocido'
        
        codigo_str = str(codigo)
        
        # Categorías principales basadas en códigos CIE-10
        if codigo_str.startswith('A') or codigo_str.startswith('B'):
            return 'enfermedades_infecciosas'
        elif codigo_str.startswith('C') or codigo_str.startswith('D'):
            return 'neoplasias'
        elif codigo_str.startswith('E'):
            return 'enfermedades_endocrinas'
        elif codigo_str.startswith('F'):
            return 'enfermedades_mentales'
        elif codigo_str.startswith('G'):
            return 'enfermedades_nerviosas'
        elif codigo_str.startswith('H'):
            return 'enfermedades_ojos_oidos'
        elif codigo_str.startswith('I'):
            return 'enfermedades_cardiovasculares'
        elif codigo_str.startswith('J'):
            return 'enfermedades_respiratorias'
        elif codigo_str.startswith('K'):
            return 'enfermedades_digestivas'
        elif codigo_str.startswith('L'):
            return 'enfermedades_piel'
        elif codigo_str.startswith('M'):
            return 'enfermedades_musculoesqueleticas'
        elif codigo_str.startswith('N'):
            return 'enfermedades_genitourinarias'
        elif codigo_str.startswith('O'):
            return 'complicaciones_embarazo'
        elif codigo_str.startswith('P'):
            return 'afecciones_perinatales'
        elif codigo_str.startswith('Q'):
            return 'malformaciones_congenitas'
        elif codigo_str.startswith('R'):
            return 'sintomas_signos_anormales'
        elif codigo_str.startswith('S') or codigo_str.startswith('T'):
            return 'traumatismos_envenenamientos'
        elif codigo_str.startswith('U'):
            return 'causas_externas'
        elif codigo_str.startswith('V') or codigo_str.startswith('W') or codigo_str.startswith('X') or codigo_str.startswith('Y'):
            return 'causas_externas_accidentes'
        elif codigo_str.startswith('Z'):
            return 'factores_influencia_salud'
        else:
            return 'otros'

    # Aplicar categorización
    dataset_modelado['categoria_diagnostico'] = dataset_modelado[columna_diagnostico].apply(categorizar_diagnostico)
    
    # Codificar categorías de diagnóstico
    categorias_diagnostico = sorted(dataset_modelado['categoria_diagnostico'].unique())
    mapeo_categorias_diagnostico = {cat: i for i, cat in enumerate(categorias_diagnostico)}
    dataset_modelado['categoria_diagnostico_codificada'] = dataset_modelado['categoria_diagnostico'].map(mapeo_categorias_diagnostico)
    
    print(f"Categorías de diagnóstico codificadas: {len(categorias_diagnostico)}")
    for categoria, codigo in mapeo_categorias_diagnostico.items():
        print(f"  {codigo}: {categoria}")
else:
    print("⚠️ No se encontraron columnas de diagnóstico")

# Mostrar resumen final
print(f"\n📊 Dataset con variables categóricas: {dataset_modelado.shape}")
print(f"�� Nuevas columnas creadas:")
nuevas_columnas = [col for col in dataset_modelado.columns if col.endswith('_codificado') or col.startswith('categoria_')]
for col in nuevas_columnas:
    print(f"  ✅ {col}")

# Verificar que no hay valores nulos en las nuevas columnas
print(f"\n🔍 Verificación de valores nulos:")
for col in nuevas_columnas:
    nulos = dataset_modelado[col].isnull().sum()
    if nulos > 0:
        print(f"  ⚠️ {col}: {nulos} valores nulos")
    else:
        print(f"  ✅ {col}: Sin valores nulos")

=== CREACIÓN DE VARIABLES CATEGÓRICAS ===
📊 Dataset base para modelado: (1246200, 17)

📋 Columnas disponibles en dataset_modelado:
['año', 'fecha_defuncion', 'sexo', 'tipo_edad', 'edad_cantidad', 'codigo_comuna', 'comuna', 'region', 'codigo_diagnostico', 'descripcion_diagnostico', 'año_fecha', 'mes', 'dia_semana', 'trimestre', 'dia_año', 'grupo_edad', 'rango_edad']

🔢 Codificando regiones...
Regiones codificadas: 17
  0: De Aisén del Gral. C. Ibáñez del Campo
  1: De Antofagasta
  2: De Arica y Parinacota
  3: De Atacama
  4: De Coquimbo
  5: De La Araucanía
  6: De Los Lagos
  7: De Los Ríos
  8: De Magallanes y de La Antártica Chilena
  9: De Tarapacá
  10: De Valparaíso
  11: De Ñuble
  12: Del Biobío
  13: Del Libertador General Bernardo O'Higgins
  14: Del Maule
  15: Ignorada
  16: Región Metropolitana

🔢 Codificando sexo...
Sexo codificado: {'Hombre': 1, 'Mujer': 0}

🔢 Codificando rangos de edad...
Rangos de edad codificados: 22
  0: 100_mas
  1: 10_14
  2: 15_19
  3: 1_4
  4: 2

In [49]:
# Limpiar valores nulos en sexo_codificado
dataset_modelado = dataset_modelado.dropna(subset=['sexo_codificado'])
print(f"📊 Dataset después de limpieza: {dataset_modelado.shape}")

📊 Dataset después de limpieza: (1246037, 22)


In [50]:
# 8.2 Generar features temporales para análisis estacional

print("=== GENERACIÓN DE FEATURES TEMPORALES ===")

# Ya tenemos algunas variables temporales básicas, vamos a crear más features útiles
print("📅 Creando features temporales adicionales...")

# Verificar columnas temporales disponibles
print(f"\n📋 Columnas temporales disponibles:")
columnas_temporales = [col for col in dataset_modelado.columns if col in ['mes', 'dia_año', 'trimestre', 'dia_semana']]
print(f"  {columnas_temporales}")

# 1. Features cíclicos para capturar estacionalidad
print("\n�� Creando features cíclicos...")

if 'mes' in dataset_modelado.columns:
    # Mes cíclico (sinusoidal y cosinusoidal)
    dataset_modelado['mes_sin'] = np.sin(2 * np.pi * dataset_modelado['mes'] / 12)
    dataset_modelado['mes_cos'] = np.cos(2 * np.pi * dataset_modelado['mes'] / 12)
    print("✅ Features cíclicos de mes creados")

if 'dia_año' in dataset_modelado.columns:
    # Día del año cíclico
    dataset_modelado['dia_año_sin'] = np.sin(2 * np.pi * dataset_modelado['dia_año'] / 365.25)
    dataset_modelado['dia_año_cos'] = np.cos(2 * np.pi * dataset_modelado['dia_año'] / 365.25)
    print("✅ Features cíclicos de día del año creados")

if 'trimestre' in dataset_modelado.columns:
    # Trimestre cíclico
    dataset_modelado['trimestre_sin'] = np.sin(2 * np.pi * dataset_modelado['trimestre'] / 4)
    dataset_modelado['trimestre_cos'] = np.cos(2 * np.pi * dataset_modelado['trimestre'] / 4)
    print("✅ Features cíclicos de trimestre creados")

# 2. Features de día de la semana (ya tenemos 'dia_semana', vamos a codificarlo)
print("\n📅 Codificando día de la semana...")

if 'dia_semana' in dataset_modelado.columns:
    # Verificar valores únicos en dia_semana
    valores_dia_semana = sorted(dataset_modelado['dia_semana'].unique())
    print(f"Valores únicos en dia_semana: {valores_dia_semana}")
    
    # Crear mapeo dinámico basado en valores reales
    mapeo_dias_semana = {dia: i for i, dia in enumerate(valores_dia_semana)}
    dataset_modelado['dia_semana_codificado'] = dataset_modelado['dia_semana'].map(mapeo_dias_semana)
    
    # Features cíclicos para día de la semana
    dataset_modelado['dia_semana_sin'] = np.sin(2 * np.pi * dataset_modelado['dia_semana_codificado'] / 7)
    dataset_modelado['dia_semana_cos'] = np.cos(2 * np.pi * dataset_modelado['dia_semana_codificado'] / 7)
    
    print(f"✅ Día de la semana codificado: {mapeo_dias_semana}")
else:
    print("⚠️ Columna 'dia_semana' no encontrada")

# 3. Features de fin de semana y días especiales
print("\n�� Creando features de días especiales...")

if 'dia_semana' in dataset_modelado.columns:
    # Fin de semana (0 = día laboral, 1 = fin de semana)
    # CORRECCIÓN: Usar valores reales encontrados
    fin_semana_valores = ['Saturday', 'Sunday', 'Sábado', 'Domingo', 'saturday', 'sunday']
    dataset_modelado['es_fin_semana'] = dataset_modelado['dia_semana'].isin(fin_semana_valores).astype(int)
    print("✅ Feature 'es_fin_semana' creado")

if 'mes' in dataset_modelado.columns:
    # Meses de invierno/verano (para Chile: invierno = Jun-Aug, verano = Dec-Feb)
    dataset_modelado['es_invierno'] = dataset_modelado['mes'].isin([6, 7, 8]).astype(int)
    dataset_modelado['es_verano'] = dataset_modelado['mes'].isin([12, 1, 2]).astype(int)
    print("✅ Features 'es_invierno' y 'es_verano' creados")
    
    # CORRECCIÓN: Trimestre del año fiscal (para análisis de políticas)
    dataset_modelado['trimestre_fiscal'] = ((dataset_modelado['mes'] - 1) // 3) + 1
    print("✅ Feature 'trimestre_fiscal' creado")

# 4. Features de época del año
print("\n📅 Creando features de época del año...")

if 'mes' in dataset_modelado.columns:
    def obtener_epoca_año(mes):
        """
        Determina la época del año basada en el mes
        """
        if mes in [12, 1, 2]:
            return 'verano'
        elif mes in [3, 4, 5]:
            return 'otoño'
        elif mes in [6, 7, 8]:
            return 'invierno'
        else:  # 9, 10, 11
            return 'primavera'

    dataset_modelado['epoca_año'] = dataset_modelado['mes'].apply(obtener_epoca_año)
    
    # Codificar época del año
    epocas_año = ['primavera', 'verano', 'otoño', 'invierno']
    mapeo_epocas = {epoca: i for i, epoca in enumerate(epocas_año)}
    dataset_modelado['epoca_año_codificada'] = dataset_modelado['epoca_año'].map(mapeo_epocas)
    
    print("✅ Época del año codificada")

# Mostrar resumen de features creados
print(f"\n📊 Dataset con features temporales: {dataset_modelado.shape}")

# Mostrar nuevas columnas creadas
nuevas_columnas_temporales = [col for col in dataset_modelado.columns if col.endswith(('_sin', '_cos', '_codificado', 'es_', 'epoca_', 'trimestre_fiscal'))]
print(f" Nuevas columnas temporales creadas: {len(nuevas_columnas_temporales)}")
for col in nuevas_columnas_temporales:
    print(f"  ✅ {col}")

print(f"\n✅ Features temporales completados")


=== GENERACIÓN DE FEATURES TEMPORALES ===
📅 Creando features temporales adicionales...

📋 Columnas temporales disponibles:
  ['mes', 'dia_semana', 'trimestre', 'dia_año']

�� Creando features cíclicos...
✅ Features cíclicos de mes creados
✅ Features cíclicos de día del año creados
✅ Features cíclicos de trimestre creados

📅 Codificando día de la semana...
Valores únicos en dia_semana: ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
✅ Día de la semana codificado: {'Friday': 0, 'Monday': 1, 'Saturday': 2, 'Sunday': 3, 'Thursday': 4, 'Tuesday': 5, 'Wednesday': 6}

�� Creando features de días especiales...
✅ Feature 'es_fin_semana' creado
✅ Features 'es_invierno' y 'es_verano' creados
✅ Feature 'trimestre_fiscal' creado

📅 Creando features de época del año...
✅ Época del año codificada

📊 Dataset con features temporales: (1246037, 37)
 Nuevas columnas temporales creadas: 12
  ✅ sexo_codificado
  ✅ rango_edad_codificado
  ✅ mes_sin
  ✅ mes_cos
  ✅ dia_año_sin


In [51]:
# 8.3 Crear variables de tendencia para el dataset temporal unificado

print("=== CREACIÓN DE VARIABLES DE TENDENCIA ===")

# Crear dataset de tendencias basado en el dataset unificado temporal
dataset_tendencias = dataset_unificado.copy()
print(f"�� Dataset base para tendencias: {dataset_tendencias.shape}")

# 1. Promedios móviles
print("\n📈 Calculando promedios móviles...")

# Promedio móvil de 3 años para nacimientos
dataset_tendencias['nacimientos_ma3'] = dataset_tendencias['nacimientos_totales'].rolling(window=3, min_periods=1).mean()

# Promedio móvil de 5 años para nacimientos
dataset_tendencias['nacimientos_ma5'] = dataset_tendencias['nacimientos_totales'].rolling(window=5, min_periods=1).mean()

# Promedio móvil de 3 años para defunciones
dataset_tendencias['defunciones_ma3'] = dataset_tendencias['defunciones_totales'].rolling(window=3, min_periods=1).mean()

# Promedio móvil de 5 años para defunciones
dataset_tendencias['defunciones_ma5'] = dataset_tendencias['defunciones_totales'].rolling(window=5, min_periods=1).mean()

print("✅ Promedios móviles creados: MA3 y MA5 para nacimientos y defunciones")

# 2. Diferencias año a año (ya las tenemos, pero vamos a crear más)
print("\n📈 Calculando diferencias adicionales...")

# Diferencia con promedio móvil
dataset_tendencias['dif_nacimientos_ma3'] = dataset_tendencias['nacimientos_totales'] - dataset_tendencias['nacimientos_ma3']
dataset_tendencias['dif_nacimientos_ma5'] = dataset_tendencias['nacimientos_totales'] - dataset_tendencias['nacimientos_ma5']
dataset_tendencias['dif_defunciones_ma3'] = dataset_tendencias['defunciones_totales'] - dataset_tendencias['defunciones_ma3']
dataset_tendencias['dif_defunciones_ma5'] = dataset_tendencias['defunciones_totales'] - dataset_tendencias['defunciones_ma5']

print("✅ Diferencias con promedios móviles creadas")

# 3. Tasas de crecimiento
print("\n📈 Calculando tasas de crecimiento...")

# Tasa de crecimiento promedio móvil (CORREGIDO: evitar división por cero)
dataset_tendencias['pct_cambio_nacimientos_ma3'] = np.where(
    dataset_tendencias['nacimientos_ma3'] != 0,
    (dataset_tendencias['dif_nacimientos_ma3'] / dataset_tendencias['nacimientos_ma3']) * 100,
    0
)

dataset_tendencias['pct_cambio_defunciones_ma3'] = np.where(
    dataset_tendencias['defunciones_ma3'] != 0,
    (dataset_tendencias['dif_defunciones_ma3'] / dataset_tendencias['defunciones_ma3']) * 100,
    0
)

print("✅ Tasas de crecimiento calculadas")

# 4. Variables de tendencia a largo plazo (SIMPLIFICADO)
print("\n�� Calculando tendencias a largo plazo...")

# Tendencia simple (diferencia entre valores extremos de ventana)
def calcular_tendencia_simple(serie, ventana=10):
    """
    Calcula tendencia simple usando diferencia entre extremos
    """
    tendencias = []
    
    for i in range(len(serie)):
        inicio = max(0, i - ventana + 1)
        fin = i + 1
        
        if fin - inicio >= 3:
            valores_ventana = serie.iloc[inicio:fin]
            if len(valores_ventana) > 1:
                tendencia = valores_ventana.iloc[-1] - valores_ventana.iloc[0]
            else:
                tendencia = 0
        else:
            tendencia = 0
            
        tendencias.append(tendencia)
    
    return pd.Series(tendencias, index=serie.index)

# Calcular tendencias simples
dataset_tendencias['tendencia_nacimientos'] = calcular_tendencia_simple(dataset_tendencias['nacimientos_totales'])
dataset_tendencias['tendencia_defunciones'] = calcular_tendencia_simple(dataset_tendencias['defunciones_totales'])

print("✅ Tendencias calculadas")

# 5. Variables de volatilidad
print("\n📈 Calculando volatilidad...")

# Volatilidad (desviación estándar móvil)
dataset_tendencias['volatilidad_nacimientos'] = dataset_tendencias['nacimientos_totales'].rolling(window=5, min_periods=1).std()
dataset_tendencias['volatilidad_defunciones'] = dataset_tendencias['defunciones_totales'].rolling(window=5, min_periods=1).std()

# Coeficiente de variación (CORREGIDO: evitar división por cero)
dataset_tendencias['cv_nacimientos'] = np.where(
    dataset_tendencias['nacimientos_ma5'] != 0,
    dataset_tendencias['volatilidad_nacimientos'] / dataset_tendencias['nacimientos_ma5'],
    0
)

dataset_tendencias['cv_defunciones'] = np.where(
    dataset_tendencias['defunciones_ma5'] != 0,
    dataset_tendencias['volatilidad_defunciones'] / dataset_tendencias['defunciones_ma5'],
    0
)

print("✅ Volatilidad calculada")

# Mostrar resumen de variables creadas
print(f"\n📊 Dataset con variables de tendencia: {dataset_tendencias.shape}")

# Mostrar nuevas columnas creadas
nuevas_columnas_tendencia = [col for col in dataset_tendencias.columns if col.endswith(('_ma3', '_ma5', '_dif_', '_pct_', '_tendencia_', '_volatilidad_', '_cv_'))]
print(f"�� Nuevas columnas de tendencia creadas: {len(nuevas_columnas_tendencia)}")
for col in nuevas_columnas_tendencia:
    print(f"  ✅ {col}")

print(f"\n✅ Variables de tendencia completadas")

=== CREACIÓN DE VARIABLES DE TENDENCIA ===
�� Dataset base para tendencias: (50, 23)

📈 Calculando promedios móviles...
✅ Promedios móviles creados: MA3 y MA5 para nacimientos y defunciones

📈 Calculando diferencias adicionales...
✅ Diferencias con promedios móviles creadas

📈 Calculando tasas de crecimiento...
✅ Tasas de crecimiento calculadas

�� Calculando tendencias a largo plazo...
✅ Tendencias calculadas

📈 Calculando volatilidad...
✅ Volatilidad calculada

📊 Dataset con variables de tendencia: (50, 39)
�� Nuevas columnas de tendencia creadas: 10
  ✅ nacimientos_ma3
  ✅ nacimientos_ma5
  ✅ defunciones_ma3
  ✅ defunciones_ma5
  ✅ dif_nacimientos_ma3
  ✅ dif_nacimientos_ma5
  ✅ dif_defunciones_ma3
  ✅ dif_defunciones_ma5
  ✅ pct_cambio_nacimientos_ma3
  ✅ pct_cambio_defunciones_ma3

✅ Variables de tendencia completadas


In [53]:
# 8.4 Crear dataset final para modelado con todas las features (CORREGIDO)

print("=== CREACIÓN DE DATASET FINAL PARA MODELADO ===")

# Crear dataset final combinando todas las features creadas
print("�� Combinando datasets para modelado...")

# Para el dataset de defunciones detalladas (dataset_modelado)
print(f"📊 Dataset de defunciones detalladas: {dataset_modelado.shape}")

# Seleccionar columnas relevantes para modelado
columnas_modelado = [
    # Variables básicas
    'año', 'fecha_defuncion', 'edad_cantidad',
    
    # Variables categóricas codificadas
    'region_codificada', 'sexo_codificado', 'rango_edad_codificado', 'categoria_diagnostico_codificada',
    
    # Variables temporales básicas
    'mes', 'trimestre', 'dia_año',
    
    # Features cíclicos
    'mes_sin', 'mes_cos', 'dia_año_sin', 'dia_año_cos', 'trimestre_sin', 'trimestre_cos',
    'dia_semana_sin', 'dia_semana_cos',
    
    # Features de días especiales
    'es_fin_semana', 'es_invierno', 'es_verano', 'trimestre_fiscal', 'epoca_año_codificada'
]

# Crear dataset final para modelado de defunciones
dataset_final_modelado = dataset_modelado[columnas_modelado].copy()

print(f"📊 Dataset final para modelado de defunciones: {dataset_final_modelado.shape}")
print(f"📋 Columnas seleccionadas: {len(columnas_modelado)}")

# Verificar valores nulos
print(f"\n🔍 Verificando valores nulos:")
nulos_por_columna = dataset_final_modelado.isnull().sum()
columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0]
if len(columnas_con_nulos) > 0:
    print("Columnas con valores nulos:")
    for col, nulos in columnas_con_nulos.items():
        print(f"  {col}: {nulos} ({nulos/len(dataset_final_modelado)*100:.1f}%)")
else:
    print("✅ No hay valores nulos en el dataset final")

# Mostrar tipos de datos
print(f"\n📋 Tipos de datos:")
tipos_datos = dataset_final_modelado.dtypes.value_counts()
for tipo, cantidad in tipos_datos.items():
    print(f"  {tipo}: {cantidad} columnas")

# Mostrar estadísticas básicas de variables numéricas
print(f"\n📊 Estadísticas básicas de variables numéricas:")
variables_numericas = dataset_final_modelado.select_dtypes(include=[np.number]).columns
print(f"Variables numéricas: {len(variables_numericas)}")
print(dataset_final_modelado[variables_numericas].describe().round(2))

# AGREGAR: Guardar dataset final para modelado
print(f"\n�� GUARDANDO DATASET FINAL PARA MODELADO...")

# CORRECCIÓN: Usar la carpeta correcta (03_primary)
import os
carpeta_final = r"C:\ProyectoML2\proyecto-ml\data\03_primary"
if not os.path.exists(carpeta_final):
    os.makedirs(carpeta_final)
    print(f"📁 Carpeta creada: {carpeta_final}")

# Guardar dataset final para modelado de defunciones
ruta_dataset_final = os.path.join(carpeta_final, "dataset_final_modelado_defunciones.csv")
dataset_final_modelado.to_csv(ruta_dataset_final, index=False)
print(f"✅ Dataset final guardado: {ruta_dataset_final}")
print(f"📊 Dimensiones: {dataset_final_modelado.shape}")

# AGREGAR: También guardar dataset de tendencias
ruta_dataset_tendencias = os.path.join(carpeta_final, "dataset_tendencias_temporales.csv")
dataset_tendencias.to_csv(ruta_dataset_tendencias, index=False)
print(f"✅ Dataset de tendencias guardado: {ruta_dataset_tendencias}")
print(f"📊 Dimensiones: {dataset_tendencias.shape}")

# AGREGAR: Guardar mapeos de codificación
import json
mapeos_codificacion = {
    "regiones": mapeo_regiones,
    "sexo": mapeo_sexo,
    "rangos_edad": mapeo_rangos_edad,
    "categorias_diagnostico": mapeo_categorias_diagnostico,
    "dias_semana": mapeo_dias_semana,
    "epocas_año": mapeo_epocas
}

ruta_mapeos = os.path.join(carpeta_final, "mapeos_codificacion.json")
with open(ruta_mapeos, 'w', encoding='utf-8') as f:
    json.dump(mapeos_codificacion, f, ensure_ascii=False, indent=2)
print(f"✅ Mapeos de codificación guardados: {ruta_mapeos}")

# Resumen final
print(f"\n📋 RESUMEN FINAL:")
print(f"✅ Dataset final para modelado: {ruta_dataset_final}")
print(f"✅ Dataset de tendencias: {ruta_dataset_tendencias}")
print(f"✅ Mapeos de codificación: {ruta_mapeos}")
print(f"✅ Todos los archivos guardados en: {carpeta_final}")

print(f"\n🎯 PRÓXIMOS PASOS RECOMENDADOS:")
print(f"  1. Usar dataset_final_modelado_defunciones.csv para modelos de clasificación")
print(f"  2. Usar dataset_tendencias_temporales.csv para modelos de series temporales")
print(f"  3. Usar mapeos_codificacion.json para interpretar resultados")

=== CREACIÓN DE DATASET FINAL PARA MODELADO ===
�� Combinando datasets para modelado...
📊 Dataset de defunciones detalladas: (1246037, 37)
📊 Dataset final para modelado de defunciones: (1246037, 23)
📋 Columnas seleccionadas: 23

🔍 Verificando valores nulos:
✅ No hay valores nulos en el dataset final

📋 Tipos de datos:
  int64: 9 columnas
  float64: 9 columnas
  int32: 4 columnas
  datetime64[ns]: 1 columnas

📊 Estadísticas básicas de variables numéricas:
Variables numéricas: 22
              año  edad_cantidad  region_codificada  sexo_codificado  \
count  1246037.00     1246037.00         1246037.00       1246037.00   
mean      2019.18          72.22              11.52             0.53   
std          3.08          18.60               4.65             0.50   
min       2014.00           0.00               0.00             0.00   
25%       2017.00          63.00               9.00             0.00   
50%       2019.00          76.00              12.00             1.00   
75%       202

In [54]:
# 8.5 Resumen final de la preparación para modelado (CORREGIDO)

print("=== RESUMEN FINAL DE LA PREPARACIÓN PARA MODELADO ===")

# Crear resumen de todos los datasets creados
datasets_finales = {
    "dataset_final_modelado": dataset_final_modelado,
    "dataset_tendencias": dataset_tendencias,
    "dataset_modelado": dataset_modelado,
    "dataset_extendido": dataset_extendido,
    "dataset_unificado": dataset_unificado
}

print("📋 Datasets creados para diferentes tipos de análisis:")
for nombre_dataset, df in datasets_finales.items():
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Dimensiones: {df.shape}")
    print(f"   Período temporal: {df['año'].min()} - {df['año'].max()}")
    print(f"   Columnas: {len(df.columns)}")

print("\n=== FEATURES CREADAS PARA MODELADO ===")

# Variables categóricas codificadas
print("�� Variables categóricas codificadas:")
categorias_creadas = [
    "region_codificada", "sexo_codificado", "rango_edad_codificado", 
    "categoria_diagnostico_codificada", "dia_semana_codificado", "epoca_año_codificada"
]
for cat in categorias_creadas:
    if cat in dataset_final_modelado.columns:
        print(f"  ✅ {cat}")

# Features temporales cíclicos
print("\n🔄 Features temporales cíclicos:")
features_ciclicos = [
    "mes_sin", "mes_cos", "dia_año_sin", "dia_año_cos", 
    "trimestre_sin", "trimestre_cos", "dia_semana_sin", "dia_semana_cos"
]
for feat in features_ciclicos:
    if feat in dataset_final_modelado.columns:
        print(f"  ✅ {feat}")

# Features de días especiales
print("\n�� Features de días especiales:")
features_especiales = [
    "es_fin_semana", "es_invierno", "es_verano", "trimestre_fiscal"
]
for feat in features_especiales:
    if feat in dataset_final_modelado.columns:
        print(f"  ✅ {feat}")

# Variables de tendencia
print("\n📈 Variables de tendencia:")
variables_tendencia = [
    "nacimientos_ma3", "nacimientos_ma5", "defunciones_ma3", "defunciones_ma5",
    "tendencia_nacimientos", "tendencia_defunciones", "volatilidad_nacimientos", "volatilidad_defunciones"
]
for var in variables_tendencia:
    if var in dataset_tendencias.columns:
        print(f"  ✅ {var}")

print("\n=== ARCHIVOS GENERADOS PARA MODELADO ===")
# CORRECCIÓN: Usar rutas correctas
archivos_modelado = [
    "dataset_final_modelado_defunciones.csv",
    "dataset_tendencias_temporales.csv", 
    "mapeos_codificacion.json"
]

for archivo in archivos_modelado:
    print(f"📁 data/03_primary/{archivo}")

print("\n=== CASOS DE USO PARA MODELADO ===")
print("🎯 Modelado de defunciones por región y edad:")
print("   - Dataset: dataset_final_modelado_defunciones.csv")
print("   - Features: region, edad, sexo, época del año, día de la semana")
print("   - Objetivo: Predecir patrones de mortalidad")

print("\n🎯 Análisis de tendencias temporales:")
print("   - Dataset: dataset_tendencias_temporales.csv")
print("   - Features: promedios móviles, tendencias lineales, volatilidad")
print("   - Objetivo: Predecir nacimientos/defunciones futuras")

print("\n🎯 Análisis estacional:")
print("   - Features: mes_sin/cos, día_semana_sin/cos, época del año")
print("   - Objetivo: Identificar patrones estacionales")

print("\n✅ Preparación para modelado completada exitosamente")

=== RESUMEN FINAL DE LA PREPARACIÓN PARA MODELADO ===
📋 Datasets creados para diferentes tipos de análisis:

DATASET_FINAL_MODELADO:
   Dimensiones: (1246037, 23)
   Período temporal: 2014 - 2024
   Columnas: 23

DATASET_TENDENCIAS:
   Dimensiones: (50, 39)
   Período temporal: 1974 - 2023
   Columnas: 39

DATASET_MODELADO:
   Dimensiones: (1246037, 37)
   Período temporal: 2014 - 2024
   Columnas: 37

DATASET_EXTENDIDO:
   Dimensiones: (10, 35)
   Período temporal: 2014 - 2023
   Columnas: 35

DATASET_UNIFICADO:
   Dimensiones: (50, 23)
   Período temporal: 1974 - 2023
   Columnas: 23

=== FEATURES CREADAS PARA MODELADO ===
�� Variables categóricas codificadas:
  ✅ region_codificada
  ✅ sexo_codificado
  ✅ rango_edad_codificado
  ✅ categoria_diagnostico_codificada
  ✅ epoca_año_codificada

🔄 Features temporales cíclicos:
  ✅ mes_sin
  ✅ mes_cos
  ✅ dia_año_sin
  ✅ dia_año_cos
  ✅ trimestre_sin
  ✅ trimestre_cos
  ✅ dia_semana_sin
  ✅ dia_semana_cos

�� Features de días especiales:
  ✅

In [56]:
# 8.6 Normalización de escalas para comparaciones entre variables

print("=== NORMALIZACIÓN DE ESCALAS ===")

# Importar librerías para normalización
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

print("📊 Normalizando escalas para comparaciones entre variables...")

# 1. Identificar variables que necesitan normalización
print("\n🔍 Identificando variables para normalización...")

# Variables numéricas en el dataset de modelado
variables_numericas_modelado = dataset_final_modelado.select_dtypes(include=[np.number]).columns.tolist()
print(f"Variables numéricas en dataset de modelado: {len(variables_numericas_modelado)}")
print(f"Variables: {variables_numericas_modelado}")

# Variables numéricas en el dataset de tendencias
variables_numericas_tendencias = dataset_tendencias.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nVariables numéricas en dataset de tendencias: {len(variables_numericas_tendencias)}")

# 2. Normalización del dataset de modelado
print("\n📊 Normalizando dataset de modelado...")

# Crear copia para normalización
dataset_modelado_normalizado = dataset_final_modelado.copy()

# Variables a normalizar (excluir códigos categóricos que ya están normalizados)
variables_a_normalizar_modelado = [col for col in variables_numericas_modelado 
                                 if col not in ['region_codificada', 'sexo_codificado', 'rango_edad_codificado', 
                                               'categoria_diagnostico_codificada', 'dia_semana_codificado', 
                                               'epoca_año_codificada', 'es_fin_semana', 'es_invierno', 'es_verano']]

print(f"Variables a normalizar en modelado: {variables_a_normalizar_modelado}")

# Aplicar StandardScaler (media=0, desviación=1)
scaler_modelado = StandardScaler()
dataset_modelado_normalizado[variables_a_normalizar_modelado] = scaler_modelado.fit_transform(
    dataset_modelado_normalizado[variables_a_normalizar_modelado]
)

print("✅ Dataset de modelado normalizado con StandardScaler")

# 3. Normalización del dataset de tendencias
print("\n📊 Normalizando dataset de tendencias...")

# Crear copia para normalización
dataset_tendencias_normalizado = dataset_tendencias.copy()

# Variables a normalizar (excluir año y variables ya normalizadas)
variables_a_normalizar_tendencias = [col for col in variables_numericas_tendencias 
                                   if col not in ['año'] and not col.endswith('_codificado')]

print(f"Variables a normalizar en tendencias: {len(variables_a_normalizar_tendencias)}")

# Aplicar diferentes tipos de normalización
# StandardScaler para la mayoría de variables
scaler_tendencias_std = StandardScaler()
dataset_tendencias_normalizado[variables_a_normalizar_tendencias] = scaler_tendencias_std.fit_transform(
    dataset_tendencias_normalizado[variables_a_normalizar_tendencias]
)

print("✅ Dataset de tendencias normalizado con StandardScaler")

# 4. Crear versiones con diferentes tipos de normalización para comparación
print("\n📊 Creando versiones con diferentes tipos de normalización...")

# MinMaxScaler (escala 0-1)
dataset_tendencias_minmax = dataset_tendencias.copy()
scaler_minmax = MinMaxScaler()
dataset_tendencias_minmax[variables_a_normalizar_tendencias] = scaler_minmax.fit_transform(
    dataset_tendencias_minmax[variables_a_normalizar_tendencias]
)

# RobustScaler (resistente a outliers)
dataset_tendencias_robust = dataset_tendencias.copy()
scaler_robust = RobustScaler()
dataset_tendencias_robust[variables_a_normalizar_tendencias] = scaler_robust.fit_transform(
    dataset_tendencias_robust[variables_a_normalizar_tendencias]
)

print("✅ Versiones creadas con MinMaxScaler y RobustScaler")

# 5. Mostrar comparación de escalas antes y después
print("\n📊 Comparación de escalas antes y después de normalización:")
print("Variables principales en dataset de tendencias:")

variables_principales = ['nacimientos_totales', 'defunciones_totales', 'crecimiento_natural', 'ratio_nacimientos_sexo']

for var in variables_principales:
    if var in dataset_tendencias.columns:
        print(f"\n{var.upper()}:")
        print(f"  Original - Min: {dataset_tendencias[var].min():.2f}, Max: {dataset_tendencias[var].max():.2f}, Media: {dataset_tendencias[var].mean():.2f}")
        print(f"  StandardScaler - Min: {dataset_tendencias_normalizado[var].min():.2f}, Max: {dataset_tendencias_normalizado[var].max():.2f}, Media: {dataset_tendencias_normalizado[var].mean():.2f}")
        print(f"  MinMaxScaler - Min: {dataset_tendencias_minmax[var].min():.2f}, Max: {dataset_tendencias_minmax[var].max():.2f}, Media: {dataset_tendencias_minmax[var].mean():.2f}")
        print(f"  RobustScaler - Min: {dataset_tendencias_robust[var].min():.2f}, Max: {dataset_tendencias_robust[var].max():.2f}, Media: {dataset_tendencias_robust[var].mean():.2f}")

print(f"\n📊 Datasets normalizados creados:")
print(f"  - dataset_modelado_normalizado: {dataset_modelado_normalizado.shape}")
print(f"  - dataset_tendencias_normalizado: {dataset_tendencias_normalizado.shape}")
print(f"  - dataset_tendencias_minmax: {dataset_tendencias_minmax.shape}")
print(f"  - dataset_tendencias_robust: {dataset_tendencias_robust.shape}")


=== NORMALIZACIÓN DE ESCALAS ===
📊 Normalizando escalas para comparaciones entre variables...

🔍 Identificando variables para normalización...
Variables numéricas en dataset de modelado: 22
Variables: ['año', 'edad_cantidad', 'region_codificada', 'sexo_codificado', 'rango_edad_codificado', 'categoria_diagnostico_codificada', 'mes', 'trimestre', 'dia_año', 'mes_sin', 'mes_cos', 'dia_año_sin', 'dia_año_cos', 'trimestre_sin', 'trimestre_cos', 'dia_semana_sin', 'dia_semana_cos', 'es_fin_semana', 'es_invierno', 'es_verano', 'trimestre_fiscal', 'epoca_año_codificada']

Variables numéricas en dataset de tendencias: 39

📊 Normalizando dataset de modelado...
Variables a normalizar en modelado: ['año', 'edad_cantidad', 'mes', 'trimestre', 'dia_año', 'mes_sin', 'mes_cos', 'dia_año_sin', 'dia_año_cos', 'trimestre_sin', 'trimestre_cos', 'dia_semana_sin', 'dia_semana_cos', 'trimestre_fiscal']
✅ Dataset de modelado normalizado con StandardScaler

📊 Normalizando dataset de tendencias...
Variables a no

In [57]:
# 8.7 Guardar datasets normalizados (CORREGIDO)

print("=== GUARDANDO DATASETS NORMALIZADOS ===")

# CORRECCIÓN: Usar carpeta 03_primary
carpeta_normalizados = r"C:\ProyectoML2\proyecto-ml\data\03_primary"

# Guardar datasets normalizados
ruta_modelado_normalizado = os.path.join(carpeta_normalizados, "dataset_modelado_defunciones_normalizado.csv")
dataset_modelado_normalizado.to_csv(ruta_modelado_normalizado, index=False)
print(f"✅ Dataset de modelado normalizado guardado: {ruta_modelado_normalizado}")

ruta_tendencias_normalizado = os.path.join(carpeta_normalizados, "dataset_tendencias_normalizado.csv")
dataset_tendencias_normalizado.to_csv(ruta_tendencias_normalizado, index=False)
print(f"✅ Dataset de tendencias normalizado guardado: {ruta_tendencias_normalizado}")

ruta_tendencias_minmax = os.path.join(carpeta_normalizados, "dataset_tendencias_minmax.csv")
dataset_tendencias_minmax.to_csv(ruta_tendencias_minmax, index=False)
print(f"✅ Dataset de tendencias MinMax guardado: {ruta_tendencias_minmax}")

ruta_tendencias_robust = os.path.join(carpeta_normalizados, "dataset_tendencias_robust.csv")
dataset_tendencias_robust.to_csv(ruta_tendencias_robust, index=False)
print(f"✅ Dataset de tendencias Robust guardado: {ruta_tendencias_robust}")

# Guardar scalers para uso posterior
import pickle

# Guardar scalers
scalers_info = {
    'scaler_modelado': scaler_modelado,
    'scaler_tendencias_std': scaler_tendencias_std,
    'scaler_minmax': scaler_minmax,
    'scaler_robust': scaler_robust,
    'variables_modelado': variables_a_normalizar_modelado,
    'variables_tendencias': variables_a_normalizar_tendencias
}

ruta_scalers = os.path.join(carpeta_normalizados, "scalers_normalizacion.pkl")
with open(ruta_scalers, 'wb') as f:
    pickle.dump(scalers_info, f)
print(f"✅ Scalers guardados: {ruta_scalers}")

# Verificar archivos guardados
archivos_normalizados = [
    ruta_modelado_normalizado,
    ruta_tendencias_normalizado,
    ruta_tendencias_minmax,
    ruta_tendencias_robust,
    ruta_scalers
]

print(f"\n�� Verificando archivos normalizados:")
for archivo in archivos_normalizados:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo) / (1024*1024)
        print(f"  ✅ {os.path.basename(archivo)}: {tamaño:.2f} MB")
    else:
        print(f"  ❌ {os.path.basename(archivo)}: No encontrado")

print(f"\n�� RESUMEN DE ARCHIVOS NORMALIZADOS:")
print(f"✅ Todos los archivos guardados en: {carpeta_normalizados}")
print(f"✅ 4 datasets normalizados + 1 archivo de scalers")

print("\n✅ Normalización de escalas completada")

=== GUARDANDO DATASETS NORMALIZADOS ===
✅ Dataset de modelado normalizado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_modelado_defunciones_normalizado.csv
✅ Dataset de tendencias normalizado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_tendencias_normalizado.csv
✅ Dataset de tendencias MinMax guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_tendencias_minmax.csv
✅ Dataset de tendencias Robust guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_tendencias_robust.csv
✅ Scalers guardados: C:\ProyectoML2\proyecto-ml\data\03_primary\scalers_normalizacion.pkl

�� Verificando archivos normalizados:
  ✅ dataset_modelado_defunciones_normalizado.csv: 364.55 MB
  ✅ dataset_tendencias_normalizado.csv: 0.03 MB
  ✅ dataset_tendencias_minmax.csv: 0.03 MB
  ✅ dataset_tendencias_robust.csv: 0.03 MB
  ✅ scalers_normalizacion.pkl: 0.01 MB

�� RESUMEN DE ARCHIVOS NORMALIZADOS:
✅ Todos los archivos guardados en: C:\ProyectoML2\proyecto-ml\data\03_primar

In [58]:
# 8.9 Actualizar resumen final con normalización (CORREGIDO)

print("=== RESUMEN FINAL ACTUALIZADO CON NORMALIZACIÓN ===")

# Actualizar resumen de datasets creados
datasets_finales_completos = {
    "dataset_final_modelado": dataset_final_modelado,
    "dataset_modelado_normalizado": dataset_modelado_normalizado,
    "dataset_tendencias": dataset_tendencias,
    "dataset_tendencias_normalizado": dataset_tendencias_normalizado,
    "dataset_tendencias_minmax": dataset_tendencias_minmax,
    "dataset_tendencias_robust": dataset_tendencias_robust,
    "dataset_extendido": dataset_extendido,
    "dataset_unificado": dataset_unificado
}

print("📋 Datasets finales creados (incluyendo versiones normalizadas):")
for nombre_dataset, df in datasets_finales_completos.items():
    print(f"\n{nombre_dataset.upper()}:")
    print(f"   Dimensiones: {df.shape}")
    print(f"   Período temporal: {df['año'].min()} - {df['año'].max()}")
    print(f"   Columnas: {len(df.columns)}")

print("\n=== TIPOS DE NORMALIZACIÓN IMPLEMENTADOS ===")

print("📊 StandardScaler (media=0, desviación=1):")
print("   ✅ dataset_modelado_normalizado")
print("   ✅ dataset_tendencias_normalizado")
print("   🎯 Uso: Algoritmos que asumen distribución normal (regresión lineal, SVM)")

print("\n📊 MinMaxScaler (escala 0-1):")
print("   ✅ dataset_tendencias_minmax")
print("   🎯 Uso: Algoritmos sensibles a la escala (redes neuronales, k-means)")

print("\n📊 RobustScaler (resistente a outliers):")
print("   ✅ dataset_tendencias_robust")
print("   🎯 Uso: Datos con outliers significativos")

print("\n=== ARCHIVOS GENERADOS COMPLETOS ===")
# CORRECCIÓN: Usar rutas correctas
archivos_completos = [
    "dataset_final_modelado_defunciones.csv",
    "dataset_modelado_defunciones_normalizado.csv",
    "dataset_tendencias_temporales.csv",
    "dataset_tendencias_normalizado.csv",
    "dataset_tendencias_minmax.csv",
    "dataset_tendencias_robust.csv",
    "mapeos_codificacion.json",
    "scalers_normalizacion.pkl"
]

for archivo in archivos_completos:
    print(f"�� data/03_primary/{archivo}")

print("\n=== BENEFICIOS DE LA NORMALIZACIÓN ===")
print("✅ Comparaciones justas entre variables de diferentes magnitudes")
print("✅ Mejor rendimiento en algoritmos de Machine Learning")
print("✅ Reducción del sesgo hacia variables con escalas mayores")
print("✅ Facilita la interpretación de coeficientes en modelos")
print("✅ Mejora la convergencia en algoritmos iterativos")

print("\n=== RECOMENDACIONES DE USO ===")
print("�� Para regresión lineal y SVM: usar StandardScaler")
print("🎯 Para redes neuronales: usar MinMaxScaler")
print("🎯 Para datos con outliers: usar RobustScaler")
print("�� Para comparaciones visuales: usar MinMaxScaler")

print("\n✅ Preparación completa para modelado con normalización terminada")


=== RESUMEN FINAL ACTUALIZADO CON NORMALIZACIÓN ===
📋 Datasets finales creados (incluyendo versiones normalizadas):

DATASET_FINAL_MODELADO:
   Dimensiones: (1246037, 23)
   Período temporal: 2014 - 2024
   Columnas: 23

DATASET_MODELADO_NORMALIZADO:
   Dimensiones: (1246037, 23)
   Período temporal: -1.6796752733960778 - 1.56487523619281
   Columnas: 23

DATASET_TENDENCIAS:
   Dimensiones: (50, 39)
   Período temporal: 1974 - 2023
   Columnas: 39

DATASET_TENDENCIAS_NORMALIZADO:
   Dimensiones: (50, 39)
   Período temporal: 1974 - 2023
   Columnas: 39

DATASET_TENDENCIAS_MINMAX:
   Dimensiones: (50, 39)
   Período temporal: 1974 - 2023
   Columnas: 39

DATASET_TENDENCIAS_ROBUST:
   Dimensiones: (50, 39)
   Período temporal: 1974 - 2023
   Columnas: 39

DATASET_EXTENDIDO:
   Dimensiones: (10, 35)
   Período temporal: 2014 - 2023
   Columnas: 35

DATASET_UNIFICADO:
   Dimensiones: (50, 23)
   Período temporal: 1974 - 2023
   Columnas: 23

=== TIPOS DE NORMALIZACIÓN IMPLEMENTADOS ===
📊 S

## 9. Manejo Avanzado de Códigos CIE-10

Esta sección se enfoca en validar y mejorar el manejo de códigos de diagnóstico CIE-10 para análisis más robustos.


In [59]:
# 9.1 Validación de códigos CIE-10

print("=== VALIDACIÓN DE CÓDIGOS CIE-10 ===")

# Analizar códigos de diagnóstico en el dataset
print("🔍 Analizando códigos de diagnóstico...")

# Obtener códigos únicos
codigos_unicos = defunciones_estandarizado['codigo_diagnostico'].unique()
print(f"📊 Total de códigos únicos encontrados: {len(codigos_unicos)}")

# Mostrar algunos ejemplos
print(f"\n📋 Primeros 10 códigos encontrados:")
for i, codigo in enumerate(sorted(codigos_unicos)[:10]):
    print(f"  {i+1:2d}. {codigo}")

# Analizar patrones de códigos
print(f"\n🔍 Análisis de patrones de códigos:")

# Códigos que empiezan con letras (códigos CIE-10 válidos)
codigos_con_letra = [codigo for codigo in codigos_unicos if pd.notna(codigo) and str(codigo)[0].isalpha()]
print(f"  Códigos que empiezan con letra: {len(codigos_con_letra)}")

# Códigos que empiezan con números
codigos_con_numero = [codigo for codigo in codigos_unicos if pd.notna(codigo) and str(codigo)[0].isdigit()]
print(f"  Códigos que empiezan con número: {len(codigos_con_numero)}")

# Códigos nulos o vacíos
codigos_nulos = [codigo for codigo in codigos_unicos if pd.isna(codigo)]
print(f"  Códigos nulos: {len(codigos_nulos)}")

# Códigos con formato extraño
codigos_raros = [codigo for codigo in codigos_unicos if pd.notna(codigo) and not str(codigo)[0].isalnum()]
print(f"  Códigos con formato extraño: {len(codigos_raros)}")

if len(codigos_raros) > 0:
    print(f"    Ejemplos: {codigos_raros[:5]}")

# Verificar códigos CIE-10 válidos
print(f"\n✅ Validación de códigos CIE-10:")
letras_validas_cie10 = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

codigos_validos = []
codigos_invalidos = []

for codigo in codigos_con_letra:
    if str(codigo)[0] in letras_validas_cie10:
        codigos_validos.append(codigo)
    else:
        codigos_invalidos.append(codigo)

print(f"  Códigos CIE-10 válidos: {len(codigos_validos)}")
print(f"  Códigos CIE-10 inválidos: {len(codigos_invalidos)}")

if len(codigos_invalidos) > 0:
    print(f"    Códigos inválidos: {codigos_invalidos[:5]}")

# Mostrar distribución por letra inicial
print(f"\n📊 Distribución por letra inicial:")
distribucion_letras = {}
for codigo in codigos_validos:
    letra = str(codigo)[0]
    distribucion_letras[letra] = distribucion_letras.get(letra, 0) + 1

for letra in sorted(distribucion_letras.keys()):
    print(f"  {letra}: {distribucion_letras[letra]:,} códigos")


=== VALIDACIÓN DE CÓDIGOS CIE-10 ===
🔍 Analizando códigos de diagnóstico...
📊 Total de códigos únicos encontrados: 20

📋 Primeros 10 códigos encontrados:
   1. A00-B99
   2. C00-D48
   3. D50-D89
   4. E00-E90
   5. F00-F99
   6. G00-G99
   7. H00-H59
   8. H60-H95
   9. I00-I99
  10. J00-J99

🔍 Análisis de patrones de códigos:
  Códigos que empiezan con letra: 20
  Códigos que empiezan con número: 0
  Códigos nulos: 0
  Códigos con formato extraño: 0

✅ Validación de códigos CIE-10:
  Códigos CIE-10 válidos: 20
  Códigos CIE-10 inválidos: 0

📊 Distribución por letra inicial:
  A: 1 códigos
  C: 1 códigos
  D: 1 códigos
  E: 1 códigos
  F: 1 códigos
  G: 1 códigos
  H: 2 códigos
  I: 1 códigos
  J: 1 códigos
  K: 1 códigos
  L: 1 códigos
  M: 1 códigos
  N: 1 códigos
  O: 1 códigos
  P: 1 códigos
  Q: 1 códigos
  R: 1 códigos
  S: 1 códigos
  U: 1 códigos


In [60]:
# 9.2 Mejorar categorización de códigos CIE-10

print("=== MEJORA DE CATEGORIZACIÓN CIE-10 ===")

# Crear función mejorada de categorización
def categorizar_diagnostico_mejorado(codigo):
    """
    Categoriza códigos CIE-10 en grupos principales mejorados
    """
    if pd.isna(codigo):
        return 'desconocido'
    
    codigo_str = str(codigo).upper()
    
    # Categorías principales mejoradas basadas en CIE-10
    if codigo_str.startswith(('A', 'B')):
        return 'enfermedades_infecciosas_parasitarias'
    elif codigo_str.startswith(('C', 'D')):
        return 'neoplasias_hematologicas'
    elif codigo_str.startswith('E'):
        return 'enfermedades_endocrinas_nutricionales'
    elif codigo_str.startswith('F'):
        return 'enfermedades_mentales_conductuales'
    elif codigo_str.startswith('G'):
        return 'enfermedades_sistema_nervioso'
    elif codigo_str.startswith('H'):
        return 'enfermedades_ojos_oidos'
    elif codigo_str.startswith('I'):
        return 'enfermedades_cardiovasculares'
    elif codigo_str.startswith('J'):
        return 'enfermedades_respiratorias'
    elif codigo_str.startswith('K'):
        return 'enfermedades_digestivas'
    elif codigo_str.startswith('L'):
        return 'enfermedades_piel_tejido_subcutaneo'
    elif codigo_str.startswith('M'):
        return 'enfermedades_musculoesqueleticas'
    elif codigo_str.startswith('N'):
        return 'enfermedades_genitourinarias'
    elif codigo_str.startswith('O'):
        return 'complicaciones_embarazo_parto'
    elif codigo_str.startswith('P'):
        return 'afecciones_perinatales'
    elif codigo_str.startswith('Q'):
        return 'malformaciones_congenitas'
    elif codigo_str.startswith('R'):
        return 'sintomas_signos_anormales'
    elif codigo_str.startswith(('S', 'T')):
        return 'traumatismos_envenenamientos'
    elif codigo_str.startswith('U'):
        return 'causas_externas_especiales'
    elif codigo_str.startswith(('V', 'W', 'X', 'Y')):
        return 'causas_externas_accidentes'
    elif codigo_str.startswith('Z'):
        return 'factores_influencia_salud'
    else:
        return 'otros_no_clasificados'

# Aplicar categorización mejorada
print("🔧 Aplicando categorización mejorada...")
defunciones_estandarizado['categoria_diagnostico_mejorada'] = defunciones_estandarizado['codigo_diagnostico'].apply(categorizar_diagnostico_mejorado)

# Mostrar distribución de categorías mejoradas
print(f"\n📊 Distribución de categorías mejoradas:")
distribucion_categorias = defunciones_estandarizado['categoria_diagnostico_mejorada'].value_counts()
for categoria, cantidad in distribucion_categorias.items():
    porcentaje = (cantidad / len(defunciones_estandarizado)) * 100
    print(f"  {categoria:35s}: {cantidad:8,} ({porcentaje:5.1f}%)")

# Identificar categorías con pocos casos
print(f"\n🔍 Categorías con pocos casos (<1% del total):")
total_registros = len(defunciones_estandarizado)
categorias_pocos_casos = distribucion_categorias[distribucion_categorias < total_registros * 0.01]

if len(categorias_pocos_casos) > 0:
    for categoria, cantidad in categorias_pocos_casos.items():
        porcentaje = (cantidad / total_registros) * 100
        print(f"  {categoria:35s}: {cantidad:8,} ({porcentaje:5.1f}%)")
else:
    print("  ✅ Todas las categorías tienen al menos 1% de los casos")

# Crear categorías de alto nivel (agrupación adicional)
print(f"\n📊 Creando categorías de alto nivel...")

def crear_categoria_alto_nivel(categoria_detallada):
    """
    Crea categorías de alto nivel agrupando categorías similares
    """
    mapeo_alto_nivel = {
        'enfermedades_infecciosas_parasitarias': 'enfermedades_infecciosas',
        'neoplasias_hematologicas': 'cancer_neoplasias',
        'enfermedades_endocrinas_nutricionales': 'enfermedades_cronicas',
        'enfermedades_mentales_conductuales': 'salud_mental',
        'enfermedades_sistema_nervioso': 'enfermedades_neurologicas',
        'enfermedades_ojos_oidos': 'enfermedades_sentidos',
        'enfermedades_cardiovasculares': 'enfermedades_cardiovasculares',
        'enfermedades_respiratorias': 'enfermedades_respiratorias',
        'enfermedades_digestivas': 'enfermedades_digestivas',
        'enfermedades_piel_tejido_subcutaneo': 'enfermedades_piel',
        'enfermedades_musculoesqueleticas': 'enfermedades_musculoesqueleticas',
        'enfermedades_genitourinarias': 'enfermedades_genitourinarias',
        'complicaciones_embarazo_parto': 'complicaciones_embarazo',
        'afecciones_perinatales': 'afecciones_perinatales',
        'malformaciones_congenitas': 'malformaciones_congenitas',
        'sintomas_signos_anormales': 'sintomas_no_especificados',
        'traumatismos_envenenamientos': 'traumatismos_accidentes',
        'causas_externas_especiales': 'causas_externas',
        'causas_externas_accidentes': 'causas_externas',
        'factores_influencia_salud': 'factores_salud',
        'otros_no_clasificados': 'otros',
        'desconocido': 'desconocido'
    }
    
    return mapeo_alto_nivel.get(categoria_detallada, 'otros')

# Aplicar categorización de alto nivel
defunciones_estandarizado['categoria_alto_nivel'] = defunciones_estandarizado['categoria_diagnostico_mejorada'].apply(crear_categoria_alto_nivel)

# Mostrar distribución de categorías de alto nivel
print(f"\n📊 Distribución de categorías de alto nivel:")
distribucion_alto_nivel = defunciones_estandarizado['categoria_alto_nivel'].value_counts()
for categoria, cantidad in distribucion_alto_nivel.items():
    porcentaje = (cantidad / len(defunciones_estandarizado)) * 100
    print(f"  {categoria:25s}: {cantidad:8,} ({porcentaje:5.1f}%)")

print(f"\n✅ Categorización mejorada completada")


=== MEJORA DE CATEGORIZACIÓN CIE-10 ===
🔧 Aplicando categorización mejorada...

📊 Distribución de categorías mejoradas:
  enfermedades_cardiovasculares      :  314,610 ( 25.2%)
  neoplasias_hematologicas           :  308,859 ( 24.8%)
  enfermedades_respiratorias         :  129,914 ( 10.4%)
  enfermedades_digestivas            :   89,496 (  7.2%)
  traumatismos_envenenamientos       :   84,968 (  6.8%)
  causas_externas_especiales         :   57,783 (  4.6%)
  enfermedades_endocrinas_nutricionales:   56,280 (  4.5%)
  enfermedades_sistema_nervioso      :   47,920 (  3.8%)
  enfermedades_genitourinarias       :   37,398 (  3.0%)
  sintomas_signos_anormales          :   34,995 (  2.8%)
  enfermedades_infecciosas_parasitarias:   26,605 (  2.1%)
  enfermedades_mentales_conductuales :   26,146 (  2.1%)
  enfermedades_piel_tejido_subcutaneo:    8,324 (  0.7%)
  malformaciones_congenitas          :    8,230 (  0.7%)
  enfermedades_musculoesqueleticas   :    7,146 (  0.6%)
  afecciones_perinata

## 10. Creación de Índices para Consultas Rápidas

Esta sección se enfoca en crear índices en los datasets principales para acelerar consultas futuras.


In [61]:
# 10.1 Crear índices en datasets principales (CORREGIDO)

print("=== CREACIÓN DE ÍNDICES PARA CONSULTAS RÁPIDAS ===")

# Función para crear índices en un dataset
def crear_indices_dataset(df, nombre_dataset):
    """
    Crea índices en un dataset para acelerar consultas comunes
    NOTA: En pandas, los índices son temporales y no persisten en CSV
    """
    print(f"\n�� Preparando índices para {nombre_dataset}...")
    
    # Crear copia para trabajar
    df_indexado = df.copy()
    
    # Verificar columnas disponibles
    columnas_disponibles = df_indexado.columns.tolist()
    print(f"  Columnas disponibles: {len(columnas_disponibles)}")
    
    # Crear índices compuestos útiles (sin resetear)
    indices_creados = []
    
    # Índice por año (más común)
    if 'año' in columnas_disponibles:
        df_indexado = df_indexado.set_index('año')
        indices_creados.append('año')
        print(f"  ✅ Índice creado: año")
    
    # Índices específicos para dataset de defunciones
    if 'region' in columnas_disponibles and 'año' in columnas_disponibles:
        df_indexado = df_indexado.reset_index()  # Resetear para crear índice compuesto
        df_indexado = df_indexado.set_index(['año', 'region'])
        indices_creados.append('año-región')
        print(f"  ✅ Índice compuesto creado: año-región")
    
    if 'sexo' in columnas_disponibles and 'año' in columnas_disponibles:
        df_indexado = df_indexado.reset_index()
        df_indexado = df_indexado.set_index(['año', 'sexo'])
        indices_creados.append('año-sexo')
        print(f"  ✅ Índice compuesto creado: año-sexo")
    
    if 'fecha_defuncion' in columnas_disponibles:
        df_indexado = df_indexado.reset_index()
        df_indexado = df_indexado.set_index('fecha_defuncion')
        indices_creados.append('fecha_defuncion')
        print(f"  ✅ Índice creado: fecha_defuncion")
    
    print(f"  📋 Índices creados: {indices_creados}")
    return df_indexado

# Crear índices en datasets principales
print("🔧 Aplicando índices a datasets principales...")

# 1. Dataset unificado temporal
dataset_unificado_indexado = crear_indices_dataset(dataset_unificado, "dataset_unificado")

# 2. Dataset extendido
dataset_extendido_indexado = crear_indices_dataset(dataset_extendido, "dataset_extendido")

# 3. Dataset de defunciones detalladas
dataset_defunciones_indexado = crear_indices_dataset(defunciones_estandarizado, "defunciones_estandarizado")

# 4. Dataset de tendencias
dataset_tendencias_indexado = crear_indices_dataset(dataset_tendencias, "dataset_tendencias")

# 5. Dataset de modelado
dataset_modelado_indexado = crear_indices_dataset(dataset_final_modelado, "dataset_modelado")

print(f"\n✅ Índices creados en todos los datasets principales")

# Mostrar información de índices creados
datasets_indexados = {
    "dataset_unificado_indexado": dataset_unificado_indexado,
    "dataset_extendido_indexado": dataset_extendido_indexado,
    "dataset_defunciones_indexado": dataset_defunciones_indexado,
    "dataset_tendencias_indexado": dataset_tendencias_indexado,
    "dataset_modelado_indexado": dataset_modelado_indexado
}

print(f"\n📋 Resumen de datasets indexados:")
for nombre, df in datasets_indexados.items():
    print(f"  {nombre}: {df.shape}")
    print(f"    Índice: {df.index.name if hasattr(df.index, 'name') else 'MultiIndex' if isinstance(df.index, pd.MultiIndex) else 'RangeIndex'}")

# NOTA IMPORTANTE
print(f"\n⚠️ NOTA IMPORTANTE:")
print(f"Los índices de pandas son temporales y no persisten al guardar en CSV.")
print(f"Para índices persistentes, se necesitaría una base de datos (SQLite, PostgreSQL, etc.)")
print(f"Los índices actuales mejoran el rendimiento de consultas en memoria.")

=== CREACIÓN DE ÍNDICES PARA CONSULTAS RÁPIDAS ===
🔧 Aplicando índices a datasets principales...

�� Preparando índices para dataset_unificado...
  Columnas disponibles: 23
  ✅ Índice creado: año
  📋 Índices creados: ['año']

�� Preparando índices para dataset_extendido...
  Columnas disponibles: 35
  ✅ Índice creado: año
  📋 Índices creados: ['año']

�� Preparando índices para defunciones_estandarizado...
  Columnas disponibles: 19
  ✅ Índice creado: año
  ✅ Índice compuesto creado: año-región
  ✅ Índice compuesto creado: año-sexo
  ✅ Índice creado: fecha_defuncion
  📋 Índices creados: ['año', 'año-región', 'año-sexo', 'fecha_defuncion']

�� Preparando índices para dataset_tendencias...
  Columnas disponibles: 39
  ✅ Índice creado: año
  📋 Índices creados: ['año']

�� Preparando índices para dataset_modelado...
  Columnas disponibles: 23
  ✅ Índice creado: año
  ✅ Índice creado: fecha_defuncion
  📋 Índices creados: ['año', 'fecha_defuncion']

✅ Índices creados en todos los datasets pr

In [63]:
# 10.2 Demostrar beneficios de los índices con consultas de ejemplo (CORREGIDO)

print("=== DEMOSTRACIÓN DE BENEFICIOS DE ÍNDICES ===")

# Crear función para medir tiempo de consulta
import time

def medir_tiempo_consulta(funcion, *args, **kwargs):
    """
    Mide el tiempo de ejecución de una consulta
    """
    inicio = time.time()
    resultado = funcion(*args, **kwargs)
    fin = time.time()
    return resultado, fin - inicio

# Ejemplos de consultas comunes que se beneficiarán de los índices
print("🔍 Ejemplos de consultas que se beneficiarán de los índices:")

# 1. Consulta por año específico (CORREGIDO)
print(f"\n�� Consulta 1: Filtrar por año 2023")

def consulta_por_año_sin_indice(df, año):
    return df[df['año'] == año]

def consulta_por_año_con_indice(df, año):
    # CORRECCIÓN: Usar el índice directamente
    return df.loc[año]

# Medir tiempo en dataset sin índice
resultado_sin_indice, tiempo_sin_indice = medir_tiempo_consulta(consulta_por_año_sin_indice, dataset_unificado, 2023)
print(f"  Dataset sin índice: {tiempo_sin_indice:.4f} segundos")

# Medir tiempo en dataset con índice (CORREGIDO)
try:
    resultado_con_indice, tiempo_con_indice = medir_tiempo_consulta(consulta_por_año_con_indice, dataset_unificado_indexado, 2023)
    print(f"  Dataset con índice: {tiempo_con_indice:.4f} segundos")
    
    if tiempo_sin_indice > 0:
        mejora = ((tiempo_sin_indice - tiempo_con_indice) / tiempo_sin_indice) * 100
        print(f"  Mejora: {mejora:.1f}%")
except KeyError:
    print(f"  Dataset con índice: Año 2023 no encontrado en el índice")

# 2. Consulta por región específica (CORREGIDO)
print(f"\n�� Consulta 2: Filtrar por región 'Región Metropolitana'")

def consulta_por_region(df, region):
    # Verificar si la columna existe
    if 'region' in df.columns:
        return df[df['region'] == region]
    else:
        return pd.DataFrame()  # Retornar DataFrame vacío si no existe la columna

resultado_region, tiempo_region = medir_tiempo_consulta(consulta_por_region, dataset_defunciones_indexado, 'Región Metropolitana')
print(f"  Tiempo de consulta por región: {tiempo_region:.4f} segundos")
print(f"  Registros encontrados: {len(resultado_region):,}")

# 3. Consulta por rango de edad (CORREGIDO)
print(f"\n📊 Consulta 3: Filtrar por rango de edad '50_mas'")

def consulta_por_edad(df, rango):
    if 'rango_edad' in df.columns:
        return df[df['rango_edad'] == rango]
    else:
        return pd.DataFrame()

resultado_edad, tiempo_edad = medir_tiempo_consulta(consulta_por_edad, dataset_defunciones_indexado, '50_mas')
print(f"  Tiempo de consulta por edad: {tiempo_edad:.4f} segundos")
print(f"  Registros encontrados: {len(resultado_edad):,}")

# 4. Consulta combinada (CORREGIDO)
print(f"\n�� Consulta 4: Filtrar por año 2023 Y región 'Región Metropolitana'")

def consulta_combinada(df, año, region):
    if 'region' in df.columns:
        return df[(df.index == año) & (df['region'] == region)]
    else:
        return pd.DataFrame()

resultado_combinada, tiempo_combinada = medir_tiempo_consulta(consulta_combinada, dataset_defunciones_indexado, 2023, 'Región Metropolitana')
print(f"  Tiempo de consulta combinada: {tiempo_combinada:.4f} segundos")
print(f"  Registros encontrados: {len(resultado_combinada):,}")

print(f"\n✅ Demostración de beneficios de índices completada")

# Mostrar tipos de consultas que se beneficiarán
print(f"\n📋 Tipos de consultas que se beneficiarán de los índices:")
consultas_beneficiadas = [
    "Filtrar por año específico (usando .loc[])",
    "Filtrar por región específica", 
    "Filtrar por rango de edad",
    "Filtrar por sexo",
    "Filtrar por categoría de diagnóstico",
    "Consultas combinadas (índice + columna)",
    "Agrupaciones por año (usando índice)",
    "Agrupaciones por región",
    "Análisis temporales por período"
]

for i, consulta in enumerate(consultas_beneficiadas, 1):
    print(f"  {i:2d}. {consulta}")

print(f"\n�� Beneficios de los índices:")
print(f"  ✅ Consultas más rápidas por año (usando .loc[])")
print(f"  ✅ Mejor rendimiento en análisis temporales")
print(f"  ✅ Agrupaciones más eficientes")
print(f"  ✅ Facilita análisis exploratorios")
print(f"  ✅ Reduce tiempo de procesamiento en modelos")

print(f"\n⚠️ NOTA IMPORTANTE:")
print(f"Cuando se usa set_index(), la columna se convierte en índice.")
print(f"Para consultas por año, usar: df.loc[año] en lugar de df[df['año'] == año]")


=== DEMOSTRACIÓN DE BENEFICIOS DE ÍNDICES ===
🔍 Ejemplos de consultas que se beneficiarán de los índices:

�� Consulta 1: Filtrar por año 2023
  Dataset sin índice: 0.0010 segundos
  Dataset con índice: 0.0000 segundos
  Mejora: 100.0%

�� Consulta 2: Filtrar por región 'Región Metropolitana'
  Tiempo de consulta por región: 0.1506 segundos
  Registros encontrados: 470,652

📊 Consulta 3: Filtrar por rango de edad '50_mas'
  Tiempo de consulta por edad: 0.0500 segundos
  Registros encontrados: 0

�� Consulta 4: Filtrar por año 2023 Y región 'Región Metropolitana'
  Tiempo de consulta combinada: 0.1053 segundos
  Registros encontrados: 0

✅ Demostración de beneficios de índices completada

📋 Tipos de consultas que se beneficiarán de los índices:
   1. Filtrar por año específico (usando .loc[])
   2. Filtrar por región específica
   3. Filtrar por rango de edad
   4. Filtrar por sexo
   5. Filtrar por categoría de diagnóstico
   6. Consultas combinadas (índice + columna)
   7. Agrupacione

In [64]:
# 10.3 Guardar datasets con índices y resumen final completo (CORREGIDO)

print("=== GUARDANDO DATASETS CON ÍNDICES ===")

# CORRECCIÓN: Usar carpeta 03_primary
carpeta_indexados = r"C:\ProyectoML2\proyecto-ml\data\03_primary"

# Guardar datasets indexados (CON ÍNDICES)
ruta_unificado_indexado = os.path.join(carpeta_indexados, "dataset_unificado_indexado.csv")
dataset_unificado_indexado.to_csv(ruta_unificado_indexado, index=True)  # CORRECCIÓN: index=True
print(f"✅ Dataset unificado indexado guardado: {ruta_unificado_indexado}")

ruta_extendido_indexado = os.path.join(carpeta_indexados, "dataset_extendido_indexado.csv")
dataset_extendido_indexado.to_csv(ruta_extendido_indexado, index=True)
print(f"✅ Dataset extendido indexado guardado: {ruta_extendido_indexado}")

ruta_defunciones_indexado = os.path.join(carpeta_indexados, "dataset_defunciones_indexado.csv")
dataset_defunciones_indexado.to_csv(ruta_defunciones_indexado, index=True)
print(f"✅ Dataset defunciones indexado guardado: {ruta_defunciones_indexado}")

ruta_tendencias_indexado = os.path.join(carpeta_indexados, "dataset_tendencias_indexado.csv")
dataset_tendencias_indexado.to_csv(ruta_tendencias_indexado, index=True)
print(f"✅ Dataset tendencias indexado guardado: {ruta_tendencias_indexado}")

ruta_modelado_indexado = os.path.join(carpeta_indexados, "dataset_modelado_indexado.csv")
dataset_modelado_indexado.to_csv(ruta_modelado_indexado, index=True)
print(f"✅ Dataset modelado indexado guardado: {ruta_modelado_indexado}")

# Guardar dataset con categorías CIE-10 mejoradas
ruta_defunciones_cie10 = os.path.join(carpeta_indexados, "dataset_defunciones_cie10_mejorado.csv")
defunciones_estandarizado.to_csv(ruta_defunciones_cie10, index=False)  # Este no tiene índices
print(f"✅ Dataset con categorías CIE-10 mejoradas guardado: {ruta_defunciones_cie10}")

# Verificar archivos guardados
archivos_indexados = [
    ruta_unificado_indexado,
    ruta_extendido_indexado,
    ruta_defunciones_indexado,
    ruta_tendencias_indexado,
    ruta_modelado_indexado,
    ruta_defunciones_cie10
]

print(f"\n Verificando archivos indexados:")
for archivo in archivos_indexados:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo) / (1024*1024)
        print(f"  ✅ {os.path.basename(archivo)}: {tamaño:.2f} MB")
    else:
        print(f"  ❌ {os.path.basename(archivo)}: No encontrado")

print(f"\n✅ Guardado de datasets indexados completado")

# RESUMEN FINAL COMPLETO
print(f"\n=== RESUMEN FINAL COMPLETO DEL PROYECTO ===")

print(f"\n�� DATASETS CREADOS:")
print(f"  ✅ Dataset unificado temporal: 50 años (1974-2023)")
print(f"  ✅ Dataset extendido con edad: 10 años (2014-2023)")
print(f"  ✅ Dataset defunciones detalladas: 1.2M registros (2014-2024)")
print(f"  ✅ Dataset de tendencias: 50 años con variables de tendencia")
print(f"  ✅ Dataset final para modelado: 1.2M registros con 23 features")

print(f"\n🔧 FEATURES IMPLEMENTADAS:")
print(f"  ✅ Variables categóricas codificadas: 5")
print(f"  ✅ Features temporales cíclicos: 8")
print(f"  ✅ Features de días especiales: 4")
print(f"  ✅ Variables de tendencia: 8")
print(f"  ✅ 3 tipos de normalización: StandardScaler, MinMaxScaler, RobustScaler")

print(f"\n📁 ARCHIVOS GENERADOS EN 03_PRIMARY:")
print(f"  ✅ 8 datasets para modelado")
print(f"  ✅ 3 tipos de normalización")
print(f"  ✅ Mapeos de codificación")
print(f"  ✅ Scalers para normalización")
print(f"  ✅ 5 datasets indexados")

print(f"\n🎯 CASOS DE USO:")
print(f"  ✅ Modelado de defunciones por región y edad")
print(f"  ✅ Análisis de tendencias temporales")
print(f"  ✅ Análisis estacional con features cíclicos")
print(f"  ✅ Clasificación de causas de muerte")
print(f"  ✅ Predicción de patrones demográficos")

print(f"\n✅ PROYECTO DE PREPARACIÓN DE DATOS COMPLETADO EXITOSAMENTE")

=== GUARDANDO DATASETS CON ÍNDICES ===
✅ Dataset unificado indexado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_unificado_indexado.csv
✅ Dataset extendido indexado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_extendido_indexado.csv
✅ Dataset defunciones indexado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_defunciones_indexado.csv
✅ Dataset tendencias indexado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_tendencias_indexado.csv
✅ Dataset modelado indexado guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_modelado_indexado.csv
✅ Dataset con categorías CIE-10 mejoradas guardado: C:\ProyectoML2\proyecto-ml\data\03_primary\dataset_defunciones_cie10_mejorado.csv

 Verificando archivos indexados:
  ✅ dataset_unificado_indexado.csv: 0.01 MB
  ✅ dataset_extendido_indexado.csv: 0.00 MB
  ✅ dataset_defunciones_indexado.csv: 244.28 MB
  ✅ dataset_tendencias_indexado.csv: 0.02 MB
  ✅ dataset_modelado_indexado.csv: 222.62 M

In [65]:
# 10.4 Resumen final completo del proyecto

print("=== RESUMEN FINAL COMPLETO DEL PROYECTO ===")

print(f"\n📊 RESUMEN DE TODAS LAS TAREAS COMPLETADAS:")

tareas_completadas = [
    "✅ 1. Limpieza crítica del dataset defunciones_filtradas",
    "✅ 2. Estandarización de nombres de columnas", 
    "✅ 3. Validación de rangos de edad",
    "✅ 4. Integración de datasets",
    "✅ 5. Validación de consistencia",
    "✅ 6. Preparación para modelado",
    "✅ 7. Normalización de escalas",
    "✅ 8. Manejo avanzado de códigos CIE-10",
    "✅ 9. Creación de índices para consultas rápidas"
]

for tarea in tareas_completadas:
    print(f"  {tarea}")

print(f"\n�� ARCHIVOS GENERADOS (TOTAL: 17 archivos):")

archivos_totales = [
    "📄 dataset_unificado_temporal.csv",
    "📄 dataset_extendido_con_edad.csv", 
    "📄 defunciones_limpias.csv",
    "📄 dataset_modelado_defunciones.csv",
    "📄 dataset_modelado_defunciones_normalizado.csv",
    "📄 dataset_tendencias_temporales.csv",
    "📄 dataset_tendencias_normalizado.csv",
    "📄 dataset_tendencias_minmax.csv",
    "📄 dataset_tendencias_robust.csv",
    "📄 dataset_unificado_indexado.csv",
    "�� dataset_extendido_indexado.csv",
    "📄 dataset_defunciones_indexado.csv",
    "📄 dataset_tendencias_indexado.csv",
    "📄 dataset_modelado_indexado.csv",
    "📄 dataset_defunciones_cie10_mejorado.csv",
    "�� mapeos_codificacion.json",
    "📄 scalers_normalizacion.pkl"
]

for archivo in archivos_totales:
    print(f"  {archivo}")

print(f"\n🎯 CASOS DE USO IMPLEMENTADOS:")

casos_uso = [
    "🔍 Análisis exploratorio de datos (EDA)",
    "📈 Análisis de tendencias temporales",
    "🏥 Análisis de mortalidad por región y edad",
    "📊 Análisis estacional de nacimientos y defunciones",
    "🤖 Modelado predictivo de mortalidad",
    "📉 Predicción de tendencias demográficas",
    "🏛️ Análisis para políticas públicas de salud",
    "�� Reportes automáticos de estadísticas vitales"
]

for caso in casos_uso:
    print(f"  {caso}")

print(f"\n🚀 PRÓXIMOS PASOS RECOMENDADOS:")

proximos_pasos = [
    "1. Ejecutar análisis exploratorio de datos (EDA)",
    "2. Crear visualizaciones de tendencias temporales",
    "3. Desarrollar modelos predictivos de mortalidad",
    "4. Implementar análisis de patrones estacionales",
    "5. Crear dashboard interactivo de estadísticas vitales",
    "6. Desarrollar sistema de alertas tempranas",
    "7. Generar reportes automáticos para autoridades"
]

for paso in proximos_pasos:
    print(f"  {paso}")

print(f"\n💡 BENEFICIOS LOGRADOS:")

beneficios = [
    "✅ Datos completamente limpios y validados",
    "✅ Variables categóricas codificadas para ML",
    "✅ Features temporales para análisis estacional",
    "✅ Variables de tendencia para predicciones",
    "✅ Datasets normalizados para comparaciones",
    "✅ Códigos CIE-10 categorizados y validados",
    "✅ Índices optimizados para consultas rápidas",
    "✅ Estructura de datos lista para producción"
]

for beneficio in beneficios:
    print(f"  {beneficio}")

=== RESUMEN FINAL COMPLETO DEL PROYECTO ===

📊 RESUMEN DE TODAS LAS TAREAS COMPLETADAS:
  ✅ 1. Limpieza crítica del dataset defunciones_filtradas
  ✅ 2. Estandarización de nombres de columnas
  ✅ 3. Validación de rangos de edad
  ✅ 4. Integración de datasets
  ✅ 5. Validación de consistencia
  ✅ 6. Preparación para modelado
  ✅ 7. Normalización de escalas
  ✅ 8. Manejo avanzado de códigos CIE-10
  ✅ 9. Creación de índices para consultas rápidas

�� ARCHIVOS GENERADOS (TOTAL: 17 archivos):
  📄 dataset_unificado_temporal.csv
  📄 dataset_extendido_con_edad.csv
  📄 defunciones_limpias.csv
  📄 dataset_modelado_defunciones.csv
  📄 dataset_modelado_defunciones_normalizado.csv
  📄 dataset_tendencias_temporales.csv
  📄 dataset_tendencias_normalizado.csv
  📄 dataset_tendencias_minmax.csv
  📄 dataset_tendencias_robust.csv
  📄 dataset_unificado_indexado.csv
  �� dataset_extendido_indexado.csv
  📄 dataset_defunciones_indexado.csv
  📄 dataset_tendencias_indexado.csv
  📄 dataset_modelado_indexado.csv